## Helper

### Imports

In [1]:
from manim import *
import numpy as np
from manim_slides import Slide

### Deadish

#### (commented out) Add LaTeX to Path

In [2]:
# import os 
# print(os.environ["PATH"])
# os.environ["PATH"] = os.environ["PATH"].replace(r":/mnt/c/Users/irona/AppData/Local/Programs/MiKTeX/miktex/bin/x64/", "")
# os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2025/bin/x86_64-linux'
# print(os.environ["PATH"])

# If you're having trouble with "latex not found" kind of issues, just close and reopen vscode (idk why it works, but it does)


#### Tex Preamble (only used in qcircuit slide)

In [3]:
# Preamble
preamble_path = "/home/ronakr/Documents/inplaceoracles/Inverting-a-Permutation-with-an-In-Place-Oracle/preamble.tex"
with open(preamble_path, 'r') as file:
    preamble = file.read()
myTemplate = TexTemplate()
myTemplate.add_to_preamble(preamble)
# myTemplate.add_to_preamble(r"\usepackage{ragged2e}")
# 
# myTemplate.add_to_preamble(rf"\geometry{{reset, legalpaper, textwidth={DOCUMENT_TEXT_WIDTH}in}}")

TexTemplate(_body='', tex_compiler='latex', description='', output_format='.dvi', documentclass='\\documentclass[preview]{standalone}', preamble='\\usepackage[english]{babel}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage[utf8]{inputenc}\n\n\\usepackage[letterpaper,margin=1in]{geometry}\n\\usepackage[OT1]{fontenc}\n\n\\usepackage{amsmath,amssymb,amsthm,amsfonts,latexsym,bbm,xspace,thm-restate}\n\\usepackage{graphicx,float,mathtools,braket,slantsc}\n\\usepackage{enumitem,booktabs,forest,mathdots,soul}\n\\usepackage[useregional]{datetime2}\n\\DTMusemodule{english}{en-US}\n\n\\usepackage{qcircuit,tikz}\n\n\\usepackage[colorlinks,citecolor=blue,,linkcolor=magenta,bookmarks=true]{hyperref}\n\\usepackage[capitalise]{cleveref}\n\n\\usepackage{multicol,array}\n\\usepackage{lipsum,framed}\n\n\\newtheorem{theorem}{Theorem}\n\\newtheorem*{theorem*}{Theorem}\n\\newtheorem{lemma}[theorem]{Lemma}\n\\newtheorem{claim}[theorem]{Claim}\n\\newtheorem{proposition}[theorem]{Proposition}\n\\ne

### Colorcode

In [4]:
RGB = lambda COLOR : ','.join(map(str,COLOR.to_rgb()))

In [5]:
def colorcode(s):
    for keyword in ['in-place', 'In-place', 'In-Place', 'P_f', 'P_{f^{-1}}']:
        s = s.replace(keyword, rf" {{\color{{IN_PLACE}} {keyword}}} ")
    for keyword in ['XOR', 'S_f', 'S_{f^{-1}}']:
        s = s.replace(keyword, rf" {{\color{{X_O_R}} {keyword}}} ")
    return s

### tex_setup

In [6]:
def tex_setup():
    config.tex_template.add_to_preamble(r"\usepackage{xcolor}")
    config.tex_template.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    config.tex_template.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")

tex_setup()

### Col Tex Template

In [7]:
DOCUMENT_TEXT_WIDTH = 4.85

def col_template(width=1):
    colTemplate = TexTemplate(
        documentclass=r"\documentclass[preview]{standalone}"
    )
    colTemplate.add_to_preamble(r"\usepackage{xcolor}")
    colTemplate.add_to_preamble(r"\usepackage{ragged2e}")
    colTemplate.add_to_preamble(r"\usepackage{geometry}")
    colTemplate.add_to_preamble(rf"\geometry{{textwidth={DOCUMENT_TEXT_WIDTH*width}in}}")
    colTemplate.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    colTemplate.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")
    return colTemplate

### Definition

In [8]:
class Definition(VGroup):
    def __init__(
        self,
        header,
        tex,
        box_color=None,
        **kwargs
    ):
        header = Tex(colorcode(header))
        tex = MathTex(colorcode(tex))
        content = VGroup(header, tex).arrange(DOWN)
        super().__init__(
            SurroundingRectangle(content, color=box_color, buff=0.3), 
            content,
            **kwargs
        )

### Bullets

In [9]:
BULLET_SEP = 1
BULLET_TEX_SCALE = 0.7

class Bullets(VGroup):
    
    def __init__(
        self,
        *lines, 
        slide=None, 
        width=1, 
        font_scale=1, 
        align_ref=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.colTemplate = col_template(width/font_scale)
        self.tex_scale = BULLET_TEX_SCALE*font_scale

        self.align_ref = align_ref

        self._prior_newlines = BULLET_SEP*1.5

        self.anims = []

        self.add_lines(*lines)

        if slide is not None:
            self.play(slide)
            

    def add_line(self, line):
        if isinstance(line, int):
            self._prior_newlines = BULLET_SEP * (1 + line)
            return
        if isinstance(line, Mobject):
            x = line.get_x()
            prev = self[-1] if len(self) > 0 else self.align_ref
            line.next_to(prev, self._prior_newlines * DOWN)
            line.set_x(x)
            self.add(line)
            self.anims.append([FadeIn(line)])
            return
        
        tex = line
        tabs = 0
        while tex[0] == '_':
            tabs += 1
            tex = tex[1:]
        bulletMe = False
        if tex[0] == '.':
            bulletMe = True
            tex = tex[1:]

        tex = colorcode(tex)
        tex = rf"\RaggedRight{{ {tex} }}"
        tex_obj = Tex(
            tex, 
            tex_template=self.colTemplate,
        ).scale(self.tex_scale)

        bullet = MathTex("\\bullet").scale(self.tex_scale)
        if bulletMe:
            bullet.next_to(tex_obj, LEFT).align_to(tex_obj, UP)
            bullet.shift(DOWN*0.5*bullet.height)
            tex_obj = VGroup(bullet, tex_obj)

        if self.align_ref is not None:
            prev = self[-1] if len(self) > 0 else self.align_ref
            tex_obj.next_to(prev, self._prior_newlines * DOWN)
            tex_obj.align_to(self.align_ref, LEFT)
        else:
            self.align_ref = tex_obj
        
        tex_obj.shift(RIGHT*tabs*(DEFAULT_MOBJECT_TO_MOBJECT_BUFFER + bullet.width))
            
        self._prior_newlines = BULLET_SEP

        self.add(tex_obj)
        self.anims.append([FadeIn(tex_obj)])

    def add_lines(self, *lines):
        for line in lines:
            self.add_line(line)

    def play(self, slide, animation=FadeIn):
        for anim_seq in self.anims:
            slide.play(*anim_seq)
            slide.next_slide()
        return self

### Steps

In [10]:
QUICK_RENDER_STEPS = False

class Steps(VGroup):

    def __init__(
        self,
        scale_factor=BULLET_TEX_SCALE,
        **kwargs,
    ) -> None:
        super().__init__(**kwargs)
        self.scale_factor = scale_factor
        

    def index(arr, key):
        if isinstance(key, int):
            return arr[key]
        for i in key:
            arr = arr[i]
        return arr
    
    
    def prev_cause(self):
        return None if len(self[-1]) == 1 else self[-1][0]
    
    
    def prev_result(self):
        return self[-1][-1]
    

    def add_step(
        self,
        *results,
        cause=None,
        overlap_prev=False,
        linear=False,
        **kwargs,
    ):
        results = map(colorcode, results)
        if cause is not None:
            cause = colorcode(cause)
        
        result_mobj = MathTex(*results, **kwargs).scale(self.scale_factor)
        if len(self) > 0:
            if overlap_prev:
                result_mobj.align_to(self.prev_result(), UL)
            else:
                result_mobj.next_to(self.prev_result(), DOWN).align_to(self.prev_result(), LEFT)
            if cause is None:
                cause = r"\xrightarrow{ \ \ \ }"
            elif cause == "":
                cause = None
            elif cause != '=':
                cause = f"\\xrightarrow{{ {cause} }}"

        self.add(VGroup())
        if cause is not None:
            cause_mobj = MathTex(cause).scale(self.scale_factor).next_to(result_mobj, LEFT)
            cause_mobj.shift(UP*cause_mobj.height*0.4)
            self[-1].add(cause_mobj)
        self[-1].add(result_mobj)

        if linear and len(self) > 1:
            self[-1].next_to(self[-2], RIGHT)
        
        return self

    def play(self, slide, indices=[-1], animation=FadeIn):
        if QUICK_RENDER_STEPS and animation == FadeIn:
            slide.add(*[self.index(i) for i in indices])
            return
        slide.play(*[animation(self.index(i)) for i in indices])
        slide.next_slide()

    def play_diff(self, slide, diff=[]):
        assert(len(self) >= 2)
        pre = self[-2][-1].copy()
        post = self.prev_result()
        post_cause = self.prev_cause()
        if (pre.get_corner(UL) != post.get_corner(UL)).any(): # if you're not on my corner
            slide.play(
                pre.animate.align_to(post, UP) # post UP
            )
        
        growing = len(pre) < len(post)
        common = list(range(max(len(pre), len(post))))
        for i in diff:
            common.remove(i)
        common = enumerate(common)
        if not growing:
            common = [(j,i) for (i,j) in common]
        
        if len(pre) == len(post):
            diff = []
        departing, arriving = ([],diff) if growing else (diff,[])

        slide.play(
            FadeIn(post_cause),
            *[FadeOut(pre[i]) for i in departing],
            *[Transform(pre[i], post[j]) for i,j in common],
        )
        if arriving:
            slide.play(
                *[FadeIn(post[i]) for i in arriving],
            )
        
        slide.add(self[-1])
        slide.remove(pre)
        slide.next_slide()

### StandardArrow

In [11]:
class StandardArrow(Arrow):

    def __init__(
        self,
        *args,
        stroke_width = 2,
        **kwargs,
    ) -> None:
        self.initial_tip_length = DEFAULT_ARROW_TIP_LENGTH*stroke_width/6
        super().__init__(
            *args, 
            stroke_width=stroke_width, 
            tip_length=self.initial_tip_length, 
            **kwargs
        )

    def get_default_tip_length(self) -> float:
        """Returns the default tip_length of the arrow.

        Examples
        --------

        ::

            >>> Arrow().get_default_tip_length()
            0.35
        """
        return self.initial_tip_length

    def _set_stroke_width_from_length(self):
        """Fixes stroke width. I think this can be replaced with a no-op."""
        if config.renderer == RendererType.OPENGL:
            # Mypy does not recognize that the self object in this case
            # is a OpenGLVMobject and that the set_stroke method is
            # defined here:
            # mobject/opengl/opengl_vectorized_mobject.py#L248
            self.set_stroke(  # type: ignore[call-arg]
                width=self.initial_stroke_width,
                recurse=False,
            )
        else:
            self.set_stroke(
                width=self.initial_stroke_width,
                family=False,
            )
        return self

### ShowTexIndices

In [12]:
def ShowTexIndices(self, tex):
    # Observe first level labels
    tex_ = tex.copy().next_to(tex, DOWN)
    self.add(index_labels(tex_))

    # Observe second level labels
    tex__ = tex_.copy().next_to(tex_, DOWN)
    for part in tex__:
        self.add(index_labels(part))

### FlipTransform

In [13]:
def FlipTransform(self, mobject_from=None, mobject_to=None):
    x0 = mobject_from.get_center()[0]
    self.play(Homotopy(lambda x,y,z,t : (np.cos(t*np.pi/2)*(x-x0) + x0, y + 0.1*(x-x0)*np.sin(t*np.pi/2), z), mobject_from, rate_func=rush_into, run_time=0.8))
    self.remove(mobject_from)
    self.play(Homotopy(lambda x,y,z,t : (np.sin(t*np.pi/2)*(x-x0) + x0, y - 0.1*(x-x0)*np.cos(t*np.pi/2), z), mobject_to, rate_func=rush_from, run_time=0.8))

In [14]:
# def _flip(x,y,z,t):
#     return (np.cos(t*np.pi)*x, 0.5*x*np.sin(t*np.pi)+y, z)

# def Flip(mobject=None, *vargs, **kwargs):
#     return Homotopy(_flip, mobject, *vargs, **kwargs)

### GroverQuery

In [15]:
def GroverQuery(self, bar_chart):
    def _negate(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*y*np.sin(t*np.pi), np.cos(t*np.pi)*y)
    idx = 0 # index of bar of marked element
    bar = bar_chart.bars[idx]
    self.play(Homotopy(_negate, bar, rate_func=rush_into, run_time=0.8))
    bar_chart.bars.remove(bar)
    bar_chart.bars.insert(idx, bar_chart._create_bar(idx, -bar_chart.values[idx]))
    bar_chart._update_colors()
    bar_chart.values[0] *= -1

### GroverDiffuseTransform

In [16]:
def GroverDiffuseTransform(self, bar_chart):
    avg = np.average(bar_chart.values)
    new_values = 2*avg - np.array(bar_chart.values)
    def _diffuse_from(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*(y-avg)*np.sin(t*np.pi/2), np.cos(t*np.pi/2)*(y-avg) + avg)
    def _diffuse_to(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x + 0.3*(y-avg)*np.cos(t*np.pi/2), np.sin(t*np.pi/2)*(y-avg) + avg)

    # Average
    avg_line = bar_chart.plot(lambda x : np.average(bar_chart.values), stroke_width=1)
    self.play(Create(avg_line))
    
    # Diffuse
    self.play(*[Homotopy(_diffuse_from, bar, rate_func=rush_into, run_time=0.8) for bar in bar_chart.bars])
    for i,bar in enumerate(bar_chart.bars):
        bar_chart.bars.remove(bar)
        bar_chart.bars.insert(i, bar_chart._create_bar(i, new_values[i]))
    bar_chart._update_colors()
    bar_chart.values[:len(new_values)] = new_values
    self.play(*[Homotopy(_diffuse_to, bar, rate_func=rush_from, run_time=0.8) for bar in bar_chart.bars])

    # Cleanup
    self.play(FadeOut(avg_line))

### ShiftBars

In [17]:
def ShiftBars(self, bar_chart):
    self.play(
        *[
            bar.animate.next_to(
            bar_chart.c2p(i+1 + 0.5, 0), 
            (UP if (bar_chart.values[i] >= 0) else DOWN), 
            buff=0) for i,bar in enumerate(bar_chart.bars)
        ]
    )
    bar_colors = [bar.get_color() for bar in bar_chart.bars]
    
    temp_bar = bar_chart.bars[-1].copy().next_to(bar_chart.c2p(0.5, 0), (UP if (bar_chart.values[-1] >= 0) else DOWN), buff=0)
    self.play(Transform(bar_chart.bars[-1], temp_bar, path_arc=90*DEGREES))
    last_val = bar_chart.values[-1]
    # last_color = bar_chart.bar_colors[-1]
    for i in range(len(bar_chart.bars)-1,0,-1):
        bar_chart.bars.remove(bar_chart.bars[i])
        bar_chart.bars.insert(i, bar_chart._create_bar(i, bar_chart.values[i-1]))
        bar_chart.values[i] = bar_chart.values[i-1]
    bar_chart.bars.remove(bar_chart.bars[0])
    bar_chart.bars.insert(0, bar_chart._create_bar(0, last_val))
    bar_chart.values[0] = last_val
    
    bar_chart.bar_colors = [bar_colors[-1]] + bar_colors[0:-1]
    bar_chart._update_colors()
    

### Circuit Braces

In [18]:
def CircuitBraces(ckt, left, right):
    bL = Brace(ckt, LEFT)
    bLt = bL.get_tex(left + r" \rightarrow")
    bR = Brace(ckt, RIGHT)
    bRt = bR.get_tex(r"\rightarrow " + right)
    return VGroup(bL, bLt, bR, bRt)

### OracleCircuit

In [19]:
def OracleCircuit(oracle_tex, left, right, length_tex=r"\ell", flipped=False):
    flipped = -1 if flipped else 1
    oracle_color, ckt_color = ((IN_PLACE, XOR) if oracle_tex[0] == 'P' else (XOR, IN_PLACE))

    oracle_mathtex = MathTex(oracle_tex)
    oracle_box = SurroundingRectangle(oracle_mathtex, buff=0.2, color=WHITE, fill_opacity=1, fill_color=oracle_color)
    oracle_drawing = VGroup(oracle_box, oracle_mathtex)

    ckt_outline = RoundedRectangle(corner_radius=0.25, height=4.0, width=6.5, fill_opacity=1, fill_color=ckt_color)

    ckt_input = MathTex(left + r" \rightarrow").next_to(ckt_outline, LEFT)
    ckt_output = MathTex(r"\rightarrow " + right).next_to(ckt_outline, RIGHT)
    ckt_input_and_output = VGroup(ckt_input, ckt_output)

    len_line = NumberLine(length=ckt_outline.width, x_range=[0, 1, 1]).next_to(ckt_outline, UP)
    len_label = MathTex(length_tex).next_to(len_line, UP)
    ckt_length = VGroup(len_line, len_label)
    # DoubleArrow(pt1, pt2, tip_shape_end=LineArrowTip, tip_shape_start=LineArrowTip, tip_length=0.1).next_to(ckt_outline, UP)
    
    oracles = VGroup()
    oracles.add(oracle_drawing.move_to(ckt_outline.get_center()).shift(flipped*2.3*LEFT + UP))
    oracles.add(oracles[0].copy().shift(flipped*1.5*RIGHT + DOWN))
    oracles.add(oracles[0].copy().shift(flipped*3*RIGHT))
    oracles.add(oracles[0].copy().shift(flipped*4.5*RIGHT + 2*DOWN))

    ckt = VGroup(ckt_outline, ckt_input_and_output, ckt_length, oracles)

    return ckt

### Get grov and HRY Amps

In [20]:
def get_Grover_amps(t, N=10):
    a_t = np.sin((2*t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((2*t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [21]:
def get_HRY_amps(t, N=10):
    a_t = np.sin((t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

## Tests

### Transform Sanity Check

In [202]:
%%manim -qh SanityCheck

class SanityCheck(Scene):
    def construct(self):
        s0 = MathTex(r"a")
        s1 = MathTex(r"\frac{a}{b}")
        s2 = MathTex(r"\frac{b}{a}")
        tms0 = TransformMatchingShapes(s0,s1, key_map={-6351953992051140758:8757917165807693890})
        tms = TransformMatchingShapes(s1,s2)#, key_map={-6351953992051140758:8757917165807693890})
        # s1a = tms.get_mobject_key(tms.get_mobject_parts(s1)[0])
        # s2l = tms.get_mobject_key(tms.get_mobject_parts(s2)[1])
        # tms.key_map = {s1a:s2l}
        print(tms0.get_mobject_parts(s0))
        print(tms0.get_mobject_parts(s1))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s0)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[1]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[2]))

        print(tms.get_mobject_parts(s1))
        print(tms.get_mobject_parts(s2))

        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[2]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[2]))
        self.play(tms0)
        self.play(tms)

Manim Community v0.19.0

[VMobjectFromSVGPath]
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
-762283919596033022
-762283919596033022
1344602826138598844
7681351932388990435
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
[VMobjectFromSVGPath, Rectangle, VMobjectFromSVGPath]
-762283919596033022
1344602826138598844
7681351932388990435
7681351932388990435
1344602826138598844
-762283919596033022


[09/11/25 05:29:21] INFO     Animation 0 : Partial movie file written in                   ]8;id=523442;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=372271;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/SanityCheck/2743010386_1912743637                         
                             _2358810818.mp4'                                                                      

[09/11/25 05:29:23] INFO     Animation 1 : Partial movie file written in                   ]8;id=768783;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=238772;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/SanityCheck/3790338095_1539069783                         
                             _2358810818.mp4'                                                                      

                    INFO     Combining to Movie file.                                      ]8;id=595884;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=844602;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=168513;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=896192;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/SanityCheck.mp4'                                                              
                                                                                                                   

                    INFO     Rendered SanityCheck                                                      ]8;id=163242;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=370066;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

### Text Test

In [203]:
%%manim -qh TextTest

class TextTest(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        title = Title(colorcode("Search with In-Place Queries"), tex_template=col_template())
        self.add(title)
        self.next_slide()

        # paragraph = Bullets(
        #     r"This is a long string of in-place text meant to span $P_f$ the entire width ${\color{IN_PLACE} x^2}$ of the XOR slide from one margin to the other, at least, that's the hope. Please note if that is not what is observed!", 
        #     align_ref=title
        # )

        # paragraph.play(self)

        bullets = Bullets(
            # "Hello, my name is bob",
            # "Hello, my name is bob",
            # 1,
            # "This sentence is meant to test that text wrapping works properly.",
            ". this is a bullet",
            "_this is an indented bullet \\\\ across multiple lines!! \\\\ So many lines...",
            "_.this is a nested bullet",
            "__.and another",
            "_.and another",
            "$\\phi + 2000$",
            slide=self, width=0.3, font_scale=0.5, align_ref=title
        )

Manim Community v0.19.0

[09/11/25 05:29:26] INFO     Animation 0 : Partial movie file written in                   ]8;id=997836;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=334864;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/TextTest/2743010386_2154729144_81                         
                             9777298.mp4'                                                                          

[09/11/25 05:29:28] INFO     Animation 1 : Partial movie file written in                   ]8;id=176881;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=968872;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/TextTest/3790338095_1443166991_12                         
                             48976031.mp4'                                                                         

[09/11/25 05:29:31] INFO     Animation 2 : Partial movie file written in                   ]8;id=558197;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=326140;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/TextTest/3790338095_1451076942_19                         
                             52314401.mp4'                                                                         

[09/11/25 05:29:33] INFO     Animation 3 : Partial movie file written in                   ]8;id=196299;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=510021;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/TextTest/3790338095_1443166991_42                         
                             48194740.mp4'                                                                         

...

                    INFO     Rendered TextTest                                                         ]8;id=740947;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=101075;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 12 animations                                                                  

[09/11/25 05:30:24] INFO     Generated 6 slides to                                                      ]8;id=684103;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=745831;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

[09/11/25 05:30:24] INFO     Generated 6 slides to                                                      ]8;id=286492;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=872862;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=540407;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=737638;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=645720;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=87629;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

### Shift Test

In [204]:
%%manim -qh ShiftTest

class ShiftTest(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        N = 10
        a_t, b_t = get_HRY_amps(1)

        chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        )

        self.play(FadeIn(chart))
        self.next_slide()
        ShiftBars(self, chart)
        self.next_slide()

Manim Community v0.19.0

[09/11/25 05:30:29] INFO     Animation 0 : Partial movie file written in                   ]8;id=602398;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=605667;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftTest/2743010386_1842627990_2                         
                             23132457.mp4'                                                                         

[09/11/25 05:30:33] INFO     Animation 1 : Partial movie file written in                   ]8;id=804287;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=193223;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftTest/3790338095_1443166991_3                         
                             727615114.mp4'                                                                        

[09/11/25 05:30:37] INFO     Animation 2 : Partial movie file written in                   ]8;id=85720;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=570953;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftTest/3790338095_1051639060_2                         
                             636834302.mp4'                                                                        

[09/11/25 05:30:40] INFO     Animation 3 : Partial movie file written in                   ]8;id=381384;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=80354;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftTest/3790338095_3812640940_6                         
                             75804156.mp4'                                                                         

...

                    INFO     Rendered ShiftTest                                                        ]8;id=726617;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=885849;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 5 animations                                                                   

[09/11/25 05:30:54] INFO     Generated 2 slides to                                                      ]8;id=281903;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=657081;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftTest'                           

[09/11/25 05:30:54] INFO     Generated 2 slides to                                                      ]8;id=178685;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=748998;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftTest'                           

                    INFO     Slide 'ShiftTest' configuration written in                                 ]8;id=353088;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=689505;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftTest.json'                            

                    INFO     Slide 'ShiftTest' configuration written in                                 ]8;id=139267;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=181184;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftTest.json'                            

## Overview

### Query Complexity (motivation)
- What is a problem? 
  - Input: some f, Output: some property of f. 
  - Egs: 
    - (1) Given: N-bit integer. Output: is it nonzero? (show arrow to "integer", saying f : [N] -> {0,1}, transform) 
      (show after (2)) Given: function f : [N] -> {0,1}. Output: is there an x such that f(x) = 1?
      mention this is "searching for a 1"
    - (2) Given: a shuffled deck of cards labeled {1,...,N}. Output: location of the "1" card.
      Given permutation f: [N] -> [N]. Output: x such that f(x) = 1.
- "In Complexity theory we often care about how much time the best algorithm to solve a problem would take as the size of the input (N) increases"
- "That's really hard to lower bound. The best technique we have is query complexity."
- In query cxty, we imagine the input is behind a paywall, and we need to pay an oracle every time we want to access bits of it. (oracle pic)
  - I will tell the oracle x and the oracle will tell me f(x), each request is called a query
  <!-- the label of the xth card in the deck. (exchange) -->
  - It's fairly intuitive that, for these specific problems above, N queries are necessary classically, (implying an Omega(N) lower bound on time?).
  - How do we do quantum query complexity? 
    - Our first thought might be |x> -> |f(x)>. 
      - Violates QM if f is not reversible. 
    - Solution: |x>|a> -> |x>|a + f(x)>. -self inverse -equiv to phase oracles
    - Okay, but what if f is a permutation? 
    - Then |x> -> |f(x)> is fine, called in-place queries.

### Key Question of this talk
- Given a permutation f, how do XOR queries and in-place queries to f compare?
  
  <!-- however, the ability to make multiple queries in superposition lets us find "1" with O(sqrt(N)) queries, through what's called Grover's algorithm
  - Understanding this alg is crucial to understanding our alg, so let's briefly review -->

### Table of comparisons
- If we have access to both f and f^{-1}, then both models can simulate the other
- Omega(sqrt(N)) XOR queries required to exactly simulate an in-place query
  - In this paper: Omega(sqrt(N)) in-place to XOR
- IndexErasure (state conversion) O(sqrt(N)) vs O(1)
  - states of the form |x>|f(x)> -> |0>|f(x)>
- SetComp (promise decision problem) O(N^{1/7}) vs O(1)
  - venn diagram highlighting: two large sets are the same or have large symmetric difference
- Unstructured search (search problem) O(sqrt(N)) vs conjectured Omega(N). (cross out conj, write ours) In this paper: O(sqrt(N)).
  - N shuffled cards, locate "1"
- Trashy Simon's (promise search problem) O(log(N)) vs conjectured Omega(sqrt(N)) ### TODO: verify sqrt(N) correct!

### Grover's
- Our problem: Given a shuffled deck of cards labeled [N], find 1. 
- (Show 12345 example truth table with f, circle inv of 1) 
- (transform problem) given a permutation f : [N] -> [N], find x^* := f^{-1}(1).
- (disappear truth table)
- In Grover's algorithm, we start off with a uniform superposition over all x in [N] (bar chart)
- Repeatedly alternate between the following two operations:
  - Query, which negates the amplitude on x^* (flip)
  - Grover's Diffusion operator, which flips all amplitudes about their average (draw line, flip)
  - (quickly evolve a few times until x^* peaks) after ~sqrt N iterations of this, we will have made ~sqrt N queries and the probability of seeing x^* when we measure.

### Where Grover's fails
- key step is being able to take |x> -> (-1)^{f(x) = 1} |x>
- |x>|0> -> |x>|f(x)> -> (-1)^{f(x) = 1}|x>|f(x)> (->) |x>|0>  <- (new slide) function erasure is hard
- Without being able to erase f, interference fails

### In fact, we show func erasure is hard!
- pf omitted here.

### Our Alg
- Do the math, have a box reminding people what in-place and XOR do
- |x> -> |x>|x>|0> -> |x>|f(x)>|0> -> |x*>|1>|1> + sum|x>|f(x)>|0>
- -> |x*>|1>|1> + sum|f(x)>|f(x)>|0> -> |x*>|1>|1> + sum|f(x)>|0>|0>  -> |x*>|1>|1> + sum|f(x)>|1>|0> -> |x*>|1> + sum|f(x)>|0>
- convert to bars

### Step Thru Our Alg
- show bars

### Kashefi
- 
- Key fact: perm inv LBs for both Sf [Ambainis] and Pf [FK18]

### Kashefi Pfs (3 slides)
- Inverting circuits
- Kashefi arg (|x>|a> -[Sf]- |x>|f(x)> in corner so we can flip it to drive home that Sf inv = Sf)
- Our arg (|x> -[Pf]- |f(x)> in corner so we can flip it to drive home that Pf inv = P{f_inv})
- => function erasure is hard (analog of index erasure)

### Sf Pf box
- Grovers + Kashefi => Theta(sqrt(N)) XOR to in-place
- Our Alg + our arg => Theta(sqrt(N)) in-place to XOR

### Summary
- Unstructured Search takes Theta(sqrt(N)) queries (in-place or XOR) (algorithm)
- Function Erasure takes O(1) XOR Queries and O(sqrt(N)) in-place queries (kashefi)
- More generally, Sf Pf Sfinv Pfinv all req sqrt(N) queries to simulate each other
- Future work: decision problem separations
  - candidates: simon, emb perm inv


## TitleSlide

In [205]:
%%manim -qh TitleSlide

class TitleSlide(Slide):
    def construct(self):
        # self.wait_time_between_slides = 1
        title = Tex(r"\textsc{Quantum Search with In-Place Queries}").scale(1.2).shift(2*UP)
        line = Underline(title)
        subtitle = Tex(r"Blake Holman, \textbf{Ronak Ramachandran}, and Justin Yirka").scale(0.75)
        subtitle.next_to(title, DOWN, buff=0.5)
        ronak = ImageMobject("ronak.png").scale_to_fit_height(2.5).next_to(subtitle, DOWN, buff=0.5)
        blake = ImageMobject("blake.jpg").scale_to_fit_height(2.5).next_to(ronak, LEFT, buff=1)
        justin = ImageMobject("jirka.jpg").scale_to_fit_height(2.5).next_to(ronak, RIGHT, buff=1)
        remail = Tex(r"ronakr@utexas.edu").next_to(ronak, DOWN).scale(0.5)
        bemail = Tex(r"holman14@purdue.edu").next_to(blake, DOWN).scale(0.5)
        jemail = Tex(r"justin@blanqet.net").next_to(justin, DOWN).scale(0.5)
        # self.play(Add(title), Write(subtitle), Succession(FadeIn(blake), FadeIn(ronak), FadeIn(justin), run_time=1.8))
        self.add(
            title, line, subtitle,
            blake, ronak, justin,
            bemail, remail, jemail
        )
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/11/25 05:31:03] INFO     Animation 0 : Partial movie file written in                   ]8;id=289087;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=868661;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/TitleSlide/2743010386_722531385_1                         
                             866445582.mp4'                                                                        

                    INFO     Combining to Movie file.                                      ]8;id=450171;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=986702;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=994387;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=366733;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/TitleSlide.mp4'                                                               
                                                                                                                   

                    INFO     Rendered TitleSlide                                                       ]8;id=683150;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=99222;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 1 animations                                                                   

[09/11/25 05:31:05] INFO     Generated 1 slides to                                                      ]8;id=893771;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=804017;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

[09/11/25 05:31:05] INFO     Generated 1 slides to                                                      ]8;id=586651;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=981260;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=221128;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=653509;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=57455;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=330459;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

# Introduction

## Complexity

In [206]:
%%manim -qh Complexity

class Complexity(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Introduction"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r".Search is everywhere",
            r".Search is function inversion:",
            Definition(
                r"\textsc{UnstructuredSearch}", 
                r"\text{Given a function $f$, find $f^{-1}(1)$}"
            ).scale(BULLET_TEX_SCALE).align_to(title, LEFT).shift(RIGHT*0.5),
            r"_.The domain of $f$ is your search space",
            r"_.$f$ outputs $1$ on desired elements",
            r"_.Searching is finding the pre-image of $1$",
            r".How much time it takes to solve this problem is important, but time is hard to quantify",
            r".One proxy for time is \textbf{query complexity}",
            align_ref=title, width=0.6
        )

        img = ImageMobject("search.jpg").scale_to_fit_height(3).next_to(bullets, buff=1).shift(UP+LEFT*0.5)

        bullets.anims[0] += [
            FadeIn(img), 
            FadeIn(SurroundingRectangle(img, color=WHITE.darker(0.5)))
        ]

        bullets.play(self)
        

Manim Community v0.19.0

[09/11/25 05:31:09] INFO     Animation 0 : Partial movie file written in                   ]8;id=914815;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=663051;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Complexity/2743010386_722531385_9                         
                             92435153.mp4'                                                                         

[09/11/25 05:31:12] INFO     Animation 1 : Partial movie file written in                   ]8;id=809341;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=704919;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Complexity/3790338095_1443166991_                         
                             222048023.mp4'                                                                        

[09/11/25 05:31:29] INFO     Animation 2 : Partial movie file written in                   ]8;id=420353;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=477300;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Complexity/3790338095_1404440052_                         
                             1225983219.mp4'                                                                       

[09/11/25 05:31:33] INFO     Animation 3 : Partial movie file written in                   ]8;id=123762;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=274057;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Complexity/3790338095_1443166991_                         
                             2187326488.mp4'                                                                       

...

                    INFO     Rendered Complexity                                                       ]8;id=853779;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=825903;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/11/25 05:33:27] INFO     Generated 9 slides to                                                      ]8;id=433129;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=635781;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Complexity'                          

[09/11/25 05:33:27] INFO     Generated 9 slides to                                                      ]8;id=501848;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=469018;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Complexity'                          

                    INFO     Slide 'Complexity' configuration written in                                ]8;id=266166;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=755863;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Complexity.json'                           

                    INFO     Slide 'Complexity' configuration written in                                ]8;id=816779;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=607843;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Complexity.json'                           

## QueryComplexity

In [207]:
%%manim -qh QueryComplexity

class QueryComplexity(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Query Complexity"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            ".Imagine our only way to access $f$ is as a black box",
            ".Can ask questions (queries) $x$",
            ".Receive answers $f(x)$",
            ".Each question is expensive",
            ".Everything else is free",
            1,
            r"The \textbf{query complexity} of a problem is the minimum number of queries necessary to solve the problem",
            align_ref=title, width=0.7
        )

        f = MathTex("f").set_color(BLACK)
        box = Square(side_length=1, fill_color=WHITE, fill_opacity=1)
        x = MathTex("x").next_to(box, LEFT*2.5)
        fx = MathTex("f(x)").next_to(box, RIGHT*2.5)
        VGroup(x,box,f,fx).next_to(bullets[:-1])

        a1 = StandardArrow(start=x.get_right()+RIGHT*0.1, end=box.get_left(), buff=0)
        a2 = StandardArrow(start=box.get_right(), end=fx.get_left()+LEFT*0.1, buff=0)
        
        def fadein(*mobs):
            self.play(*[(GrowArrow if isinstance(mob, StandardArrow) else FadeIn)(mob) for mob in mobs])
            self.next_slide()

        fadein(bullets[0], box, f)
        fadein(bullets[1], a1, x)
        fadein(bullets[2], a2, fx)
        for b in bullets[3:]:
            fadein(b)

Manim Community v0.19.0

[09/11/25 05:33:33] INFO     Animation 0 : Partial movie file written in                   ]8;id=969166;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=983203;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryComplexity/2743010386_722531                         
                             385_2406306580.mp4'                                                                   

[09/11/25 05:33:37] INFO     Animation 1 : Partial movie file written in                   ]8;id=29236;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=957313;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryComplexity/3790338095_144316                         
                             6991_3082818887.mp4'                                                                  

[09/11/25 05:33:42] INFO     Animation 2 : Partial movie file written in                   ]8;id=704361;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=548650;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryComplexity/3790338095_154172                         
                             2006_4262736286.mp4'                                                                  

[09/11/25 05:33:45] INFO     Animation 3 : Partial movie file written in                   ]8;id=844341;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=377000;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryComplexity/3790338095_144316                         
                             6991_1960381484.mp4'                                                                  

...

                    INFO     Rendered QueryComplexity                                                  ]8;id=922737;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=467885;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 14 animations                                                                  

[09/11/25 05:34:36] INFO     Generated 7 slides to                                                      ]8;id=114098;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=734156;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryComplexity'                     

[09/11/25 05:34:36] INFO     Generated 7 slides to                                                      ]8;id=950165;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=362146;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryComplexity'                     

                    INFO     Slide 'QueryComplexity' configuration written in                           ]8;id=693826;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=944161;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryComplexity.json'                      

                    INFO     Slide 'QueryComplexity' configuration written in                           ]8;id=53043;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=259781;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryComplexity.json'                      

## QueryModels

In [208]:
%%manim -qh QueryModels

class QueryModels(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"How can quantum computers make queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        sep = 0.5

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        ).scale(0.7)

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        ).scale(0.7)

        bullets = Bullets(
            ".To perform a query, we need to be able to query $x$ and receive back $f(x)$",
            ".Quantum mechanics is reversible, so quantum queries need to be reversible",
            r".If $f$ is not reversible, the map $|x\rangle \rightarrow |f(x)\rangle$ is not unitary",
            r"_Solution: ",
            xor_defn,
            r".When $f$ is a permutation from $[N]$ to $[N]$, then $|x\rangle \rightarrow |f(x)\rangle$ is unitary",
            in_place_defn,
            align_ref=title
        ).play(self)
        

Manim Community v0.19.0

[09/11/25 05:34:39] INFO     Animation 0 : Partial movie file written in                   ]8;id=276508;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=646410;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryModels/2743010386_722531385_                         
                             4029937755.mp4'                                                                       

[09/11/25 05:34:41] INFO     Animation 1 : Partial movie file written in                   ]8;id=572016;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=740996;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryModels/3790338095_1443166991                         
                             _1397953554.mp4'                                                                      

[09/11/25 05:34:45] INFO     Animation 2 : Partial movie file written in                   ]8;id=36695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=659963;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryModels/3790338095_886179735_                         
                             4256878999.mp4'                                                                       

[09/11/25 05:34:47] INFO     Animation 3 : Partial movie file written in                   ]8;id=744745;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=492074;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/QueryModels/3790338095_1443166991                         
                             _47221767.mp4'                                                                        

...

                    INFO     Rendered QueryModels                                                      ]8;id=354063;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=51546;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 16 animations                                                                  

[09/11/25 05:35:57] INFO     Generated 8 slides to                                                      ]8;id=518935;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=970353;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryModels'                         

[09/11/25 05:35:57] INFO     Generated 8 slides to                                                      ]8;id=887355;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=540255;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/QueryModels'                         

                    INFO     Slide 'QueryModels' configuration written in                               ]8;id=196819;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=396743;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryModels.json'                          

                    INFO     Slide 'QueryModels' configuration written in                               ]8;id=34609;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=816505;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/QueryModels.json'                          

# In-Place v. XOR

## KeyQuestion

In [209]:
%%manim -qh KeyQuestion

class KeyQuestion(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        question = Tex(
            r"Given a \textbf{permutation} $f$, how do \textbf{XOR queries} \\ and \textbf{in-place queries} to $f$ compare?",
            substrings_to_isolate=[r"\textbf{XOR queries}", r"\textbf{in-place queries}"]
        )
        question.set_color_by_tex(r"XOR", XOR)
        question.set_color_by_tex(r"in-place", IN_PLACE)
        self.play(Write(question))
        self.next_slide()

Manim Community v0.19.0

[09/11/25 05:36:04] INFO     Animation 0 : Partial movie file written in                   ]8;id=892405;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=472765;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/KeyQuestion/2743010386_2382438420                         
                             _223132457.mp4'                                                                       

[09/11/25 05:36:06] INFO     Animation 1 : Partial movie file written in                   ]8;id=542331;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=559903;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/KeyQuestion/3790338095_1443166991                         
                             _4246822100.mp4'                                                                      

                    INFO     Combining to Movie file.                                      ]8;id=544811;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=167492;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=981605;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=126957;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/KeyQuestion.mp4'                                                              
                                                                                                                   

                    INFO     Rendered KeyQuestion                                                      ]8;id=60074;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=468962;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/11/25 05:36:10] INFO     Generated 1 slides to                                                      ]8;id=850717;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=179268;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

[09/11/25 05:36:10] INFO     Generated 1 slides to                                                      ]8;id=915195;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=667618;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=277795;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=715805;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=238085;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=301458;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

## Comparisons

In [210]:
%%manim -qh Comparisons

class Comparisons(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Comparing XOR and in-place queries"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        sep = 1.5
        oracle_defns = VGroup(
            xor_defn, in_place_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep).next_to(title, DOWN*sep)
        self.play(FadeIn(oracle_defns))
        self.next_slide()

        bullets = Bullets(
            r"With access to both $f$ and $f^{-1}$, XOR and in-place queries are equivalent",
            r"With only access to $f$, in-place queries look very powerful [KKVB02]:",
            r".There is an $f$ where",
            r"_.XOR queries to $f$ are easy to implement",
            r"_.In-place queries to $f$ efficiently solve (Non-Automorphic) Graph Isomorphism",
            r"_.Simulating in-place queries with few XOR queries $\implies$ efficient algorithms",
            r".$\Omega(\sqrt{N})$ XOR queries to $f$ are necessary to simulate one in-place query to $f$",
        ).next_to(oracle_defns, DOWN*sep).align_to(title, LEFT).play(self)

Manim Community v0.19.0

[09/11/25 05:36:13] INFO     Animation 0 : Partial movie file written in                   ]8;id=839756;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=68593;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Comparisons/2743010386_722531385_                         
                             1715074805.mp4'                                                                       

[09/11/25 05:36:15] INFO     Animation 1 : Partial movie file written in                   ]8;id=218804;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=48934;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Comparisons/3790338095_1443166991                         
                             _275121098.mp4'                                                                       

[09/11/25 05:36:18] INFO     Animation 2 : Partial movie file written in                   ]8;id=721570;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=379021;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Comparisons/3790338095_161108985_                         
                             4278875779.mp4'                                                                       

[09/11/25 05:36:21] INFO     Animation 3 : Partial movie file written in                   ]8;id=800625;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=187315;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Comparisons/3790338095_3225289990                         
                             _1930564268.mp4'                                                                      

...

                    INFO     Rendered Comparisons                                                      ]8;id=814116;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=914447;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/11/25 05:37:42] INFO     Generated 9 slides to                                                      ]8;id=228737;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=673606;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Comparisons'                         

[09/11/25 05:37:42] INFO     Generated 9 slides to                                                      ]8;id=886737;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=882296;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Comparisons'                         

                    INFO     Slide 'Comparisons' configuration written in                               ]8;id=387270;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=652014;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Comparisons.json'                          

                    INFO     Slide 'Comparisons' configuration written in                               ]8;id=214152;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=423173;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Comparisons.json'                          

## LeadingCand

In [211]:
%%manim -qh LeadingCand

class LeadingCand(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given permutation $f : [N] \rightarrow [N]$, find $f^{-1}(1)$}",
        ).scale(BULLET_TEX_SCALE)

        bullets = Bullets(
            r".Prior to this work, no evidence to the contrary",
            r".However, in-place and XOR queries were generally believed to be incomparable",
            r".Leading candidate: unstructured search",
            perm_inv_defn,
            r".$O(\sqrt{N})$ XOR queries, by Grover's search algorithm [Gro96]",
            r".[FK18] conjecture $\Omega(N)$ in-place queries required",
            align_ref=title
        )
        bullets.anims.append([Create(Cross(bullets[-1]))])
        bullets.add_lines(
            r".We devise algorithm using only $O(\sqrt{N})$ in-place queries!",
        )

        bullets.play(self)


Manim Community v0.19.0

[09/11/25 05:37:46] INFO     Animation 0 : Partial movie file written in                   ]8;id=81475;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=660011;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LeadingCand/2743010386_722531385_                         
                             3769187286.mp4'                                                                       

[09/11/25 05:37:48] INFO     Animation 1 : Partial movie file written in                   ]8;id=547236;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=161099;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LeadingCand/3790338095_1443166991                         
                             _2506270670.mp4'                                                                      

[09/11/25 05:37:52] INFO     Animation 2 : Partial movie file written in                   ]8;id=271290;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=458821;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LeadingCand/3790338095_3186605611                         
                             _1855009834.mp4'                                                                      

[09/11/25 05:37:54] INFO     Animation 3 : Partial movie file written in                   ]8;id=575680;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=37238;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LeadingCand/3790338095_1443166991                         
                             _104963032.mp4'                                                                       

...

                    INFO     Rendered LeadingCand                                                      ]8;id=473839;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=565618;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/11/25 05:39:06] INFO     Generated 9 slides to                                                      ]8;id=432091;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=986164;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/LeadingCand'                         

[09/11/25 05:39:06] INFO     Generated 9 slides to                                                      ]8;id=987708;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=773429;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/LeadingCand'                         

                    INFO     Slide 'LeadingCand' configuration written in                               ]8;id=385613;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=269826;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/LeadingCand.json'                          

                    INFO     Slide 'LeadingCand' configuration written in                               ]8;id=69197;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=460530;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/LeadingCand.json'                          

## FuncErasIntro

In [212]:
%%manim -qh FuncErasIntro

class FuncErasIntro(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        ).scale(BULLET_TEX_SCALE)

        bullets = Bullets(
            r"We define a state conversion task:",
            func_erase_defn,
            r".Possible with 1 XOR query",
            r".Requires $\Omega(\sqrt{N})$ in-place queries",
            r"First known problem requiring more in-place queries than XOR queries!",
            align_ref=title
        ).play(self)
            

Manim Community v0.19.0

[09/11/25 05:39:09] INFO     Animation 0 : Partial movie file written in                   ]8;id=578006;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=464565;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/FuncErasIntro/2743010386_72253138                         
                             5_3769187286.mp4'                                                                     

[09/11/25 05:39:12] INFO     Animation 1 : Partial movie file written in                   ]8;id=336250;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=444975;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/FuncErasIntro/3790338095_14431669                         
                             91_2506270670.mp4'                                                                    

[09/11/25 05:39:14] INFO     Animation 2 : Partial movie file written in                   ]8;id=606842;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=183982;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/FuncErasIntro/3790338095_30915270                         
                             8_488607230.mp4'                                                                      

[09/11/25 05:39:17] INFO     Animation 3 : Partial movie file written in                   ]8;id=886060;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=350534;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/FuncErasIntro/3790338095_14431669                         
                             91_2612445865.mp4'                                                                    

...

                    INFO     Rendered FuncErasIntro                                                    ]8;id=394908;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=802202;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 12 animations                                                                  

[09/11/25 05:39:55] INFO     Generated 6 slides to                                                      ]8;id=204633;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=227784;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasIntro'                       

[09/11/25 05:39:55] INFO     Generated 6 slides to                                                      ]8;id=919148;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=192832;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasIntro'                       

                    INFO     Slide 'FuncErasIntro' configuration written in                             ]8;id=911697;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=814977;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasIntro.json'                        

                    INFO     Slide 'FuncErasIntro' configuration written in                             ]8;id=796744;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=505359;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasIntro.json'                        

## Plan

In [22]:
%%manim -qh Plan

class Plan(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Talk Overview"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r"1) Why does Grover's search algorithm fail with in-place queries?",
            r"_.Answer: \textsc{FunctionErasure} is hard!",
            r"2) Our search algorithm using $O(\sqrt{N})$ in-place queries",
            r"3) Our $\Omega(\sqrt{N})$ lower bound for \textsc{FunctionErasure} with in-place queries",
            align_ref=title
        ).play(self)

Manim Community v0.19.0

[09/14/25 12:38:16] INFO     Animation 0 : Partial movie file written in                   ]8;id=220885;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=633299;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Plan/2743010386_722531385_4480258                         
                             0.mp4'                                                                                

[09/14/25 12:38:29] INFO     Animation 1 : Partial movie file written in                   ]8;id=537710;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=215957;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Plan/3790338095_1443166991_336746                         
                             0938.mp4'                                                                             

[09/14/25 12:38:33] INFO     Writing \RaggedRight{ 2) Our search algorithm using            ]8;id=262755;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=262571;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\
                             $O(\sqrt{N})$  {\color{IN_PLACE} in-place}  queries } to                              
                             media/Tex/04f200a74b6f27d3.tex                                                        

[09/14/25 12:39:03] INFO     Animation 2 : Partial movie file written in                   ]8;id=162852;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=829466;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/Plan/3790338095_1500791793_422327                         
                             2008.mp4'                                                                             

...

                    INFO     Rendered Plan                                                             ]8;id=709167;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=849787;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 10 animations                                                                  

[09/14/25 12:41:08] INFO     Generated 5 slides to                                                      ]8;id=939522;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=326764;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Plan'                                

[09/14/25 12:41:08] INFO     Generated 5 slides to                                                      ]8;id=911454;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=806814;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Plan'                                

                    INFO     Slide 'Plan' configuration written in                                      ]8;id=770388;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=311238;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Plan.json'                                 

                    INFO     Slide 'Plan' configuration written in                                      ]8;id=985575;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=286337;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Plan.json'                                 

# Why Grover Fails --> FE LB

## GroverSection

In [67]:
%%manim -qh GroverSection

class GroverSection(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        section = Bullets(r"Where Grover's Search Fails", font_scale=1.5)
        secnum = VGroup(Circle(color=WHITE, radius=0.4), Tex("1")).scale(0.8).next_to(section, UP, buff=0.5)
        self.add(section, secnum)
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/14/25 11:24:17] INFO     Animation 0 : Partial movie file written in                   ]8;id=601187;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=746747;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/GroverSection/2743010386_72253138                         
                             5_468082968.mp4'                                                                      

[09/14/25 11:24:19] INFO     Animation 1 : Partial movie file written in                   ]8;id=330732;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=323392;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/GroverSection/3790338095_14431669                         
                             91_3992001669.mp4'                                                                    

                    INFO     Combining to Movie file.                                      ]8;id=560304;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=994524;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=176297;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=405075;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/GroverSection.mp4'                                                            
                                                                                                                   

                    INFO     Rendered GroverSection                                                    ]8;id=297088;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=431598;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/14/25 11:24:21] INFO     Generated 1 slides to                                                      ]8;id=122326;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=751877;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverSection'                       

[09/14/25 11:24:21] INFO     Generated 1 slides to                                                      ]8;id=104718;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=326455;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverSection'                       

                    INFO     Slide 'GroverSection' configuration written in                             ]8;id=659164;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=946458;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverSection.json'                        

                    INFO     Slide 'GroverSection' configuration written in                             ]8;id=823239;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=795344;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverSection.json'                        

## StepThruGroverBar

In [214]:
%%manim -qh StepThruGroverBar

class StepThruGroverBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        title = Title("Grover's Algorithm")
        self.add(title)
        self.wait()
        self.next_slide()

        N = 10
        T = int(np.ceil(np.pi/4 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[XOR] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1), #, "1", "$f(1)$",r"$f^{2}(1)$", r"$f^{3}(1)$", "", r"$\dots$", "", r"$f^{-2}(x^*)$", r"$f^{-1}(x^*)$"],
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)
        VGroup(chart, y_axis_label).scale(0.7).next_to(title, DOWN*2)

        query_box = Definition(
            r"Query $f$",
            r"\text{Negate $|x^*\rangle$ amplitude}",
            box_color=XOR
        )

        diffuse_box = Definition(
            r"Diffuse",
            r"\text{Flip about average}"
        ).match_height(query_box)

        op_boxes = VGroup(
            query_box, 
            diffuse_box,
        ).arrange(RIGHT).scale(0.7).next_to(chart, DOWN)

        inactive_opacity = 0.2
        inactive_darkness = 1 - inactive_opacity

        query_box[0].set_color(XOR.darker(inactive_darkness))
        query_box[1].set_opacity(inactive_opacity)
        diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
        diffuse_box[1].set_opacity(inactive_opacity)

        self.play(
            DrawBorderThenFill(chart), 
            DrawBorderThenFill(y_axis_label),
            FadeIn(op_boxes),
        )
        self.next_slide()

        a_t,b_t = get_Grover_amps(0,N)
        for t in range(1,T):
            # Query
            query_box[0].set_color(XOR)
            query_box[1].set_opacity(1)
            diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
            diffuse_box[1].set_opacity(inactive_opacity)
            
            GroverQuery(self, chart)
            self.next_slide()

            # Diffuse
            query_box[0].set_color(XOR.darker(inactive_darkness))
            query_box[1].set_opacity(inactive_opacity)
            diffuse_box[0].set_color(WHITE)
            diffuse_box[1].set_opacity(1)

            a_t,b_t = get_Grover_amps(t,N)
            GroverDiffuseTransform(self, chart)
            self.next_slide()

        query_box[0].set_color(XOR.darker(inactive_darkness))
        query_box[1].set_opacity(inactive_opacity)
        diffuse_box[0].set_color(WHITE.darker(inactive_darkness))
        diffuse_box[1].set_opacity(inactive_opacity)

        question = Bullets(r"Where does this go wrong with in-place queries?").next_to(op_boxes, DOWN)

        self.play(FadeIn(question))
        self.next_slide()

        self.play(
            query_box[0].animate.set_color(XOR),
            query_box[1].animate.set_opacity(1),
            Circumscribe(query_box[0], color=IN_PLACE, fade_out=True),
        )
        self.next_slide()


Manim Community v0.19.0

[09/11/25 05:40:37] INFO     Animation 0 : Partial movie file written in                   ]8;id=815195;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=605405;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/StepThruGroverBar/2743010386_7225                         
                             31385_4276835967.mp4'                                                                 

[09/11/25 05:40:39] INFO     Animation 1 : Partial movie file written in                   ]8;id=365601;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=805377;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/StepThruGroverBar/3790338095_1443                         
                             166991_3725775767.mp4'                                                                

[09/11/25 05:40:46] INFO     Animation 2 : Partial movie file written in                   ]8;id=803652;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=867178;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/StepThruGroverBar/3790338095_3522                         
                             273171_1371791780.mp4'                                                                

[09/11/25 05:40:48] INFO     Animation 3 : Partial movie file written in                   ]8;id=269434;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=690696;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/StepThruGroverBar/3790338095_1443                         
                             166991_4148043888.mp4'                                                                

...

                    INFO     Rendered StepThruGroverBar                                                ]8;id=866647;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=69872;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[09/11/25 05:42:24] INFO     Generated 8 slides to                                                      ]8;id=420465;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=784827;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

[09/11/25 05:42:24] INFO     Generated 8 slides to                                                      ]8;id=663599;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=75776;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=543006;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=631986;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=564752;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=949607;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

## GroverBreakdown

In [34]:
%%manim -qh GroverBreakdown

class GroverBreakdown(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Why can't we Grover's search with in-place queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        query_tex_cases = MathTex(r"|x\rangle \rightarrow \begin{cases} -|x\rangle & \text{if $f(x) = 1$} \\ |x\rangle & \text{otherwise} \end{cases}")
        query_tex = MathTex(r"|x\rangle \rightarrow (-1)^{f(x)=1}|x\rangle")
        self.play(Write(query_tex_cases))
        self.next_slide()
        self.play(ReplacementTransform(query_tex_cases, query_tex))
        self.next_slide()
        
        self.play(query_tex.animate.shift(2*UP).scale(0.8))
        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        ).scale(0.8).next_to(query_tex, DOWN)
        self.play(FadeIn(xor_defn))
        self.next_slide()
        
        steps = VGroup()
        steps.add(MathTex(
            r"|x\rangle",
            r"|0\rangle",
        ).next_to(xor_defn, DOWN))
        steps.add(MathTex(
            r"\xrightarrow{S_f}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.add(MathTex(
            r"\xrightarrow{\phantom{S_f}} (-1)^{f(x)=1}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.add(MathTex(
            r"\xrightarrow{S_f} (-1)^{f(x)=1}",
            r"|x\rangle",
            r"|0\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps[0].align_to(steps[1][1], LEFT)
        steps[1][0][0:2].set_fill(color=XOR)
        steps[3][0][0:2].set_fill(color=XOR)
        steps.scale(0.8).shift(LEFT*0.5)

        # ShowTexIndices(self, steps)

        self.play(FadeIn(steps[0][0]))
        self.next_slide()
        self.play(FadeIn(steps[0][1]))
        self.next_slide()
        self.play(FadeIn(steps[1]))
        self.next_slide()
        self.play(FadeIn(steps[2]))
        self.next_slide()
        self.play(FadeIn(steps[3][0]), FadeIn(steps[3][1]), FadeIn(steps[3][2]))
        self.next_slide()
        # self.play(FadeOut(steps[3][-1]))
        # self.next_slide()

        self.play(Circumscribe(VGroup(steps[2], steps[3]), color=RED, fade_out=True))

        func_eras = MathTex(
            r"|x\rangle|f(x)\rangle",
            r"\rightarrow",
            r"|x\rangle",
            r"|0\rangle",
        )
        self.play(
            FadeOut(query_tex, steps[0], steps[1], steps[2][0], steps[3][0], xor_defn),
            ReplacementTransform(steps[2][1], func_eras[0]),
            FadeIn(func_eras[1]),
            ReplacementTransform(steps[3][1], func_eras[2]),
            ReplacementTransform(steps[3][2], func_eras[3])
        )
        self.next_slide()

        func_eras_title = Tex(r"\textsc{FunctionErasure}").next_to(func_eras, 2*UP)
        self.play(Write(func_eras_title))
        self.next_slide()

        lower_bound = Bullets(
            r"We prove \textsc{FunctionErasure} requires $\Omega(\sqrt{N})$ in-place queries!"
        ).next_to(func_eras, DOWN*2)
        self.play(FadeIn(lower_bound))
        self.next_slide()


Manim Community v0.19.0

[09/14/25 08:34:04] INFO     Animation 0 : Using cached data (hash :                           ]8;id=752960;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=214911;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2743010386_722531385_1917479231)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=589410;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=140044;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_4129930199)                                                     

[09/14/25 08:34:05] INFO     Animation 2 : Using cached data (hash :                           ]8;id=490796;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=365261;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_4016078521_2092611628)                                                     

[09/14/25 08:34:06] INFO     Animation 3 : Using cached data (hash :                           ]8;id=770595;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=780429;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_1115947222)                                                     

...

                    INFO     Rendered GroverBreakdown                                                  ]8;id=776644;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=576552;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 26 animations                                                                  

[09/14/25 08:35:21] INFO     Generated 12 slides to                                                     ]8;id=76368;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=396098;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

[09/14/25 08:35:21] INFO     Generated 12 slides to                                                     ]8;id=310254;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=601450;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=973502;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=346509;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=595336;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=118630;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

# Search With In-Place Queries

## AlgSection

In [68]:
%%manim -qh AlgSection

class AlgSection(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        section = Bullets(r"Search with $O(\sqrt{N})$ In-Place Queries", font_scale=1.5)
        secnum = VGroup(Circle(color=WHITE, radius=0.4), Tex("2")).scale(0.8).next_to(section, UP, buff=0.5)
        self.add(section, secnum)
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/14/25 11:24:23] INFO     Animation 0 : Partial movie file written in                   ]8;id=310326;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=72745;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/AlgSection/2743010386_722531385_1                         
                             400126969.mp4'                                                                        

[09/14/25 11:24:26] INFO     Animation 1 : Partial movie file written in                   ]8;id=824976;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=152167;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/AlgSection/3790338095_1443166991_                         
                             1860414037.mp4'                                                                       

                    INFO     Combining to Movie file.                                      ]8;id=568495;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=893385;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=389956;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=885372;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/AlgSection.mp4'                                                               
                                                                                                                   

                    INFO     Rendered AlgSection                                                       ]8;id=245667;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=968383;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/14/25 11:24:31] INFO     Generated 1 slides to                                                      ]8;id=274695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=344434;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgSection'                          

[09/14/25 11:24:31] INFO     Generated 1 slides to                                                      ]8;id=706473;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=234338;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgSection'                          

                    INFO     Slide 'AlgSection' configuration written in                                ]8;id=186043;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=34884;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgSection.json'                           

                    INFO     Slide 'AlgSection' configuration written in                                ]8;id=751762;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=975993;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgSection.json'                           

## ShiftDemo

In [23]:
%%manim -qh ShiftDemo

class ShiftDemo(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title("Without phase flips, what can we do?")
        self.add(title)
        self.wait()
        self.next_slide()

        N = 10
        T = int(np.ceil(np.pi/4 * np.sqrt(N)))

        chart = BarChart(
            [0.05220109, 0.37703556, 0.36249917, 0.24089972, 0.1580988, 0.19792895, 0.12961734, 0.27102231, 0.35441177, 0.62100593],
            y_range=[-1, 1, 1],
            bar_names=[r"$x^*$", "1", "$f(1)$",r"$f^{2}(1)$", r"$f^{3}(1)$", "", r"$\dots$", "", r"$f^{-2}(x^*)$", r"$f^{-1}(x^*)$"],
            x_axis_config={"font_size": 26},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)
        VGroup(chart, y_axis_label).next_to(title, DOWN, buff=1).shift(LEFT*0.5)

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        ).scale(0.7).next_to(chart, DOWN).shift(UP*1.5)

        self.play(
            DrawBorderThenFill(chart), 
            DrawBorderThenFill(y_axis_label),
            FadeIn(in_place_defn),
        )
        self.next_slide()

        ShiftBars(self, chart)
        self.next_slide()

Manim Community v0.19.0

[09/14/25 08:03:54] INFO     Animation 0 : Partial movie file written in                   ]8;id=784402;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=867793;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftDemo/2743010386_722531385_27                         
                             09192554.mp4'                                                                         

[09/14/25 08:04:01] INFO     Animation 1 : Partial movie file written in                   ]8;id=209678;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=200565;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ShiftDemo/3790338095_1443166991_3                         
                             670402990.mp4'                                                                        

                    INFO     Writing 1 to media/Tex/d5fdb9908e88f514.tex                    ]8;id=700574;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=161993;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\

[09/14/25 08:04:06] INFO     Writing $f(1)$ to media/Tex/c0aba9bb23890562.tex               ]8;id=857713;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=652733;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\

...

                    INFO     Rendered ShiftDemo                                                        ]8;id=899550;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=277529;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 7 animations                                                                   

[09/14/25 08:05:52] INFO     Generated 3 slides to                                                      ]8;id=963424;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=221280;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftDemo'                           

[09/14/25 08:05:52] INFO     Generated 3 slides to                                                      ]8;id=429091;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=308287;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ShiftDemo'                           

                    INFO     Slide 'ShiftDemo' configuration written in                                 ]8;id=729358;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=848276;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftDemo.json'                            

                    INFO     Slide 'ShiftDemo' configuration written in                                 ]8;id=752251;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=693456;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ShiftDemo.json'                            

## OurAlg

In [22]:
%%manim -qh OurAlg

QUICK_RENDER_STEPS = False

class OurAlg(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Can we mimic one iteration of Grover's search?"))
        self.add(title)
        self.wait()
        self.next_slide()

        tech_title = Title(colorcode(r"Can we amplify $|x^*\rangle$ by $\Omega\left( \tfrac{1}{\sqrt{N}} \right)$ with $O(1)$ in-place queries?"), font_size=36)
        self.play(Transform(title, tech_title))
        self.next_slide()

        N = 10
        t = 1
        a_t, b_t = get_HRY_amps(t)

        init_chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        ).shift(LEFT*3.3).scale(0.5)

        steps = Steps(scale_factor=0.5)
        steps.add_step(
            r"\alpha |x^*\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle"
        ).next_to(title, DOWN).shift(2.5*RIGHT).play(self)

        init_chart.align_to(steps[-1], DOWN)
        self.play(DrawBorderThenFill(init_chart))
        self.next_slide()
        
        self.remove(steps[-1])
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|0\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|0\rangle",
            overlap_prev=True, cause=""
        ).play_diff(self, diff=(1,4))
        steps.remove(steps[-2])

        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|x^*\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|x\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|f(x)\rangle",
            cause=r"P_f"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |x\rangle",
            r"|f(x)\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self, diff=(2,6))

        split_sep = 1.7
        mark_chart_a = init_chart.copy()
        mark_chart_b = init_chart.copy()
        mark_chart_a.change_bar_values([a_t] + [0]*(N-1))
        mark_chart_b.change_bar_values([0] + [b_t]*(N-1))
        arrow_start = init_chart.get_bottom()
        l_arrow_end = mark_chart_a.copy().shift(LEFT*split_sep).align_to(steps[-1], DOWN).get_top()
        r_arrow_end = mark_chart_a.copy().shift(RIGHT*split_sep).align_to(steps[-1], DOWN).get_top()
        l_arrow = StandardArrow(start=arrow_start, end=l_arrow_end, buff=0.5)
        r_arrow = StandardArrow(start=arrow_start, end=r_arrow_end, buff=0.5)
        self.play(
            mark_chart_a.animate.shift(LEFT*split_sep).align_to(steps[-1], DOWN),
            mark_chart_b.animate.shift(RIGHT*split_sep).align_to(steps[-1], DOWN),
            GrowArrow(l_arrow),
            GrowArrow(r_arrow),
        )
        mark_chart_a_flag = MathTex(r"\otimes |1\rangle\ ", r"+").scale(0.5).next_to(mark_chart_a)
        mark_chart_b_flag = MathTex(r"\otimes |0\rangle").scale(0.5).next_to(mark_chart_b)
        self.play(
            FadeIn(mark_chart_a_flag),
            FadeIn(mark_chart_b_flag),
        )
        self.next_slide()

        mark_label = Tex(
            "``Mark''"
        ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center_of_mass())
        self.play(FadeIn(mark_label))
        self.next_slide()
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|1\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |f(x)\rangle",
            r"|f(x)\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{ctrl-}P_f"
        ).play_diff(self)
        
        steps.add_step(
            r"\alpha |x^*\rangle",
            r"|0\rangle",
            r"\otimes |1\rangle",
            r" + \sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"\beta |f(x)\rangle",
            r"|0\rangle",
            r"\otimes |0\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)

        shift_charts = VGroup(
            mark_chart_a.copy(), 
            mark_chart_a_flag.copy(),
            mark_chart_b.copy(),
            mark_chart_b_flag.copy(),
        )

        l_arrow_start = mark_chart_a.get_bottom()
        r_arrow_start = mark_chart_b.get_bottom()
        l_arrow_end = shift_charts[0].copy().align_to(steps[-1], DOWN).get_top()
        r_arrow_end = shift_charts[2].copy().align_to(steps[-1], DOWN).get_top()
        l_arrow = StandardArrow(start=l_arrow_start, end=l_arrow_end, buff=0.2)
        r_arrow = StandardArrow(start=r_arrow_start, end=r_arrow_end, buff=0.2)
        self.play(
            shift_charts.animate.align_to(steps[-1], DOWN),
            GrowArrow(l_arrow),
            GrowArrow(r_arrow),
        )
        self.next_slide()

        self.play(Indicate(shift_charts[2:]))
        ShiftBars(self, shift_charts[2])
        self.next_slide()

        shift_label = Tex(
            "``Shift''"
        ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
        self.play(FadeIn(shift_label))
        self.next_slide()

        # ================================================================
        #  Use -n 39 flag to render from first animation after this point
        # ================================================================

        old_copies = [title, *shift_charts]
        title = title.copy()
        shift_charts = shift_charts.copy()
        for mob in old_copies:
            self.remove(mob)
        to_remove = self.mobjects
        self.add(title)
        self.play(
            *[FadeOut(mob) for mob in to_remove],
            shift_charts.animate.next_to(title, DOWN),
        )
        for mob in to_remove:
            self.remove(mob)
        self.next_slide()

        line_scale = 0.8
        tex_scale = 0.9

        self.play(
            shift_charts[0].animate.shift(LEFT*1.5),
            shift_charts[1][0].animate.shift(LEFT*1.5),
            shift_charts[2:].animate.shift(RIGHT*1.5),
        )

        norm_a = MathTex(r"\frac{1}{\sqrt{2}}").scale(0.5).next_to(shift_charts[0], LEFT)
        norm_b = MathTex(r"\frac{1}{\sqrt{2}}").scale(0.5).next_to(shift_charts[2], LEFT)
        flag_a = MathTex(r"\otimes \left( |+\rangle - |-\rangle \right)").scale(0.5).next_to(shift_charts[0])
        flag_b = MathTex(r"\otimes \left( |+\rangle + |-\rangle \right)").scale(0.5).next_to(shift_charts[2])

        self.play(
            Transform(shift_charts[1][0], flag_a),
            Transform(shift_charts[3], flag_b),
            FadeIn(norm_a),
            FadeIn(norm_b),
        )
        self.next_slide()

        betw = Tex(
            "Let's group terms by the state of the flag in the Hadamard basis!"
        ).scale(BULLET_TEX_SCALE*tex_scale).move_to(shift_charts[1][1]).shift(DOWN*line_scale)
        self.play(FadeIn(betw))
        self.next_slide()

        ref_pos = VGroup(
            shift_charts[0].copy(),
            shift_charts[2].copy(),
            shift_charts[1][1].copy(),
        ).shift(DOWN*2*line_scale)

        b0 = shift_charts[2].copy()
        b1 = shift_charts[2].copy()
        norm_b0 = norm_b.copy()
        norm_b1 = norm_b.copy()
        flag_b0 = MathTex(r"\otimes |+\rangle").scale(0.5).next_to(b0)
        flag_b1 = MathTex(r"\otimes |-\rangle").scale(0.5).next_to(b1)
        plus = ref_pos[2]

        self.play(
            FadeIn(norm_b0),
            FadeIn(norm_b1),
            FadeIn(flag_b0),
            FadeIn(flag_b1),
            FadeIn(plus),
            b0.animate.move_to(ref_pos[0]),
            b1.animate.move_to(ref_pos[1]),
            norm_b0.animate.next_to(ref_pos[0], LEFT),
            norm_b1.animate.next_to(ref_pos[1], LEFT),
            flag_b0.animate.next_to(ref_pos[0]),
            flag_b1.animate.next_to(ref_pos[1])
        )
        self.next_slide()

        a0 = shift_charts[0].copy()
        a1 = shift_charts[0].copy()
        norm_a0 = norm_a.copy()
        norm_a1 = norm_a.copy()
        flag_a0 = MathTex(r"\otimes |+\rangle").scale(0.5).next_to(a0)
        flag_a1 = MathTex(r"\otimes |-\rangle").scale(0.5).next_to(a1)
        bar_a0 = a0.bars[0].copy()
        bar_a1 = a1.bars[0].copy()
        a0.change_bar_values([0]*N)
        a1.change_bar_values([0]*N)

        self.play(
            FadeIn(norm_a0),
            FadeIn(norm_a1),
            FadeIn(flag_a0),
            FadeIn(flag_a1),
            FadeIn(bar_a0),
            FadeIn(bar_a1),
            a0.animate.move_to(ref_pos[0]),
            a1.animate.move_to(ref_pos[1]),
            bar_a0.animate.next_to(b0.bars[0], UP, buff=0),
            bar_a1.animate.align_to(b1.bars[0], UL),
            norm_a0.animate.next_to(ref_pos[0], LEFT),
            norm_a1.animate.next_to(ref_pos[1], LEFT),
            flag_a0.animate.next_to(ref_pos[0]),
            flag_a1.animate.next_to(ref_pos[1])
        )
        b1.bar_colors = init_chart.bar_colors
        b1.change_bar_values([b_t - a_t] + [0] + [b_t]*(N-2))
        dups = [a0, a1, norm_a0, norm_a1, flag_a0, flag_a1, bar_a1]
        for mob in dups:
            self.remove(mob)
        self.next_slide()

        good = Tex(
            "Amplified!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[0], DOWN)
        self.play(FadeIn(good))
        self.next_slide()

        bad = Tex(
            "Dampened..."
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[1], DOWN)
        self.play(FadeIn(bad))
        self.next_slide()

        redemption = Tex(
            "Looks like phase query - let's Diffuse!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(ref_pos[1], DOWN)
        self.play(Transform(bad, redemption))
        self.next_slide()

        d = VGroup(
            b0,
            b1,
            norm_b0,
            norm_b1,
            flag_b0,
            flag_b1,
            bar_a0,
            plus,
        ).copy()
        
        self.play(
            d.animate.shift(DOWN*2*line_scale),
        )
        GroverDiffuseTransform(self, d[1])
        self.next_slide()

        norm_plus = MathTex(r"\frac{1}{2}").scale(0.5).next_to(d[0], LEFT)
        norm_minus = MathTex(r"\frac{1}{2}").scale(0.5).next_to(d[1], LEFT)
        flag_plus = MathTex(r"\otimes \left( |0\rangle + |1\rangle \right)").scale(0.5).next_to(d[0])
        flag_minus = MathTex(r"\otimes \left( |0\rangle - |1\rangle \right)").scale(0.5).next_to(d[1])

        self.play(
            Transform(d[2], norm_plus),
            Transform(d[3], norm_minus),
            Transform(d[4], flag_plus),
            Transform(d[5], flag_minus),
        )
        self.next_slide()
        
        betw2 = Tex(
            "Grouping terms by the state of the flag in the standard basis..."
        ).scale(BULLET_TEX_SCALE*tex_scale).move_to(d[7]).shift(DOWN*line_scale)
        self.play(FadeIn(betw2))
        self.next_slide()

        target = d.copy().shift(DOWN*2*line_scale)
        
        target[0].bar_colors = init_chart.bar_colors
        target[1].bar_colors = init_chart.bar_colors
        target[0].change_bar_values([b_t + a_t - (b_t + a_t)/N] + [b_t - (b_t + a_t)/N]*(N-1))
        target[1].change_bar_values([(b_t + a_t)/N] + [-b_t + (b_t + a_t)/N] + [(b_t + a_t)/N]*(N-2))
        f0 = MathTex(r"\otimes |0\rangle").scale(0.5)
        f1 = MathTex(r"\otimes |1\rangle").scale(0.5)
        l_target = (target[0] + target[2] + f0.copy().next_to(target[0]))
        r_target = (target[1] + target[3] + f1.copy().next_to(target[1]))
        l_source_1 = (d[0] + d[2] + f0.copy().next_to(d[0]) + d[6]).copy()
        l_source_2 = (d[0] + d[2] + f1.copy().next_to(d[0]) + d[6]).copy()
        r_source_1 = (d[1] + d[3] + f0.copy().next_to(d[1])).copy()
        r_source_2 = (d[1] + d[3] + f1.copy().next_to(d[1])).copy()
        self.play(
            l_source_1.animate.align_to(l_target, DL),
            r_source_1.animate.align_to(l_target, DL),
        )
        self.add(l_target)
        for mob in [l_source_1, r_source_1]:
            self.remove(mob)
        self.play(
            l_source_2.animate.align_to(r_target, DL),
            r_source_2.animate.align_to(r_target, DL),
        )
        self.add(r_target)
        for mob in [l_source_2, r_source_2]:
            self.remove(mob)
        self.play(FadeIn(target[-1])) # plus
        self.next_slide()

        good = Tex(
            "Amplified!"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(target[0], DOWN)
        self.play(FadeIn(good))
        self.next_slide()

        bad = Tex(
            "Negligible"
        ).scale(BULLET_TEX_SCALE*tex_scale).next_to(target[1], DOWN)
        self.play(
            Create(Cross(Rectangle(height=line_scale, width=3.5).move_to(target[1]))),
            FadeIn(bad)
        )
        self.next_slide()

Manim Community v0.19.0

[09/14/25 10:47:44] INFO     Animation 0 : Using cached data (hash :                           ]8;id=231797;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=792645;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2743010386_722531385_963382998)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=360449;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=571581;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_1528453069)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=834346;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=250909;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1442064601_1554596757)                                                     

[09/14/25 10:47:45] INFO     Animation 3 : Using cached data (hash :                           ]8;id=424425;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=882991;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_3707085253)                                                     

...

                    INFO     Rendered OurAlg                                                           ]8;id=753947;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=246205;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 75 animations                                                                  

[09/14/25 10:53:33] INFO     Generated 29 slides to                                                     ]8;id=611325;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=664824;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

[09/14/25 10:53:33] INFO     Generated 29 slides to                                                     ]8;id=521725;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=495946;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=615506;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=651344;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=828406;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=966088;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

## OurBars

In [44]:
%%manim -qh OurBars

class OurBars(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode("Search with In-Place Queries"))
        self.add(title)
        self.wait()
        self.next_slide()

        split_sep = 2
        line_scale = 1.8
        chart_scale = 0.6

        # Init

        N = 10
        t = 0
        a_t, b_t = get_HRY_amps(t)

        init_chart = BarChart(
            [a_t] + [b_t]*(N-1),
            y_range=[0, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 30},
            y_length=1,
            x_length=4,
        ).scale(chart_scale).next_to(title, DOWN)

        self.play(FadeIn(init_chart))
        self.next_slide()

        for i in (0,1):

            # Mark

            mark_chart_a = init_chart.copy()
            mark_chart_b = init_chart.copy()
            mark_chart_a.change_bar_values([a_t] + [0]*(N-1))
            mark_chart_b.change_bar_values([0] + [b_t]*(N-1))
            arrow_start = init_chart.get_bottom()
            l_arrow_end = mark_chart_a.copy().shift(LEFT*split_sep + DOWN*line_scale).get_top()
            r_arrow_end = mark_chart_a.copy().shift(RIGHT*split_sep + DOWN*line_scale).get_top()
            l_arrow = StandardArrow(start=arrow_start, end=l_arrow_end, buff=0.5)
            r_arrow = StandardArrow(start=arrow_start, end=r_arrow_end, buff=0.5)
            mark_chart_a_flag = MathTex(r"\otimes |1\rangle\ \ \ ", r"+").scale(0.5).next_to(mark_chart_a)
            plus = mark_chart_a_flag[1].shift(LEFT*split_sep + DOWN*line_scale)
            mark_chart_a_flag = mark_chart_a_flag[0]
            mark_chart_b_flag = MathTex(r"\otimes |0\rangle").scale(0.5).next_to(mark_chart_b)
            mark_label = Tex(
                "``Mark''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center_of_mass())
            self.play(
                (mark_chart_a + mark_chart_a_flag).animate.shift(LEFT*split_sep + DOWN*line_scale),
                (mark_chart_b + mark_chart_b_flag).animate.shift(RIGHT*split_sep + DOWN*line_scale),
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(mark_label),
                FadeIn(plus, lag_ratio=0.5),
            )
            self.next_slide()

            # Shift

            shift_charts = VGroup(
                mark_chart_a.copy(), 
                VGroup(mark_chart_a_flag, plus).copy(),
                mark_chart_b.copy(),
                mark_chart_b_flag.copy(),
            )

            l_arrow_start = mark_chart_a.get_bottom()
            r_arrow_start = mark_chart_b.get_bottom()
            l_arrow_end = shift_charts[0].copy().shift(DOWN*line_scale).get_top()
            r_arrow_end = shift_charts[2].copy().shift(DOWN*line_scale).get_top()
            l_arrow = StandardArrow(start=l_arrow_start, end=l_arrow_end, buff=0.2)
            r_arrow = StandardArrow(start=r_arrow_start, end=r_arrow_end, buff=0.2)
            shift_label = Tex(
                "``Shift''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
            self.play(
                shift_charts.animate.shift(DOWN*line_scale),
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(shift_label),
            )
            ShiftBars(self, shift_charts[2])
            self.next_slide()

            # Diffuse

            t += 1
            a_t, b_t = get_HRY_amps(t)

            final_chart = BarChart(
                [a_t] + [b_t]*(N-1),
                y_range=[0, 1, 1],
                bar_colors=[IN_PLACE] + [GRAY]*(N-1),
                bar_names=[r"$x^*$"] + [""]*(N-1),
                x_axis_config={"font_size": 30},
                y_length=1,
                x_length=4,
            ).scale(chart_scale).align_to(shift_charts[0], DOWN).shift(DOWN*line_scale)

            l_arrow_start = shift_charts[0].get_bottom()
            r_arrow_start = shift_charts[2].get_bottom()
            arrow_end = final_chart.get_top()
            l_arrow = StandardArrow(start=l_arrow_start, end=arrow_end+LEFT*0.3, buff=0.2)
            r_arrow = StandardArrow(start=r_arrow_start, end=arrow_end+RIGHT*0.3, buff=0.2)
            diffuse_label = Tex(
                "``Diffuse''"
            ).scale(BULLET_TEX_SCALE).move_to(VGroup(l_arrow, r_arrow).get_center())
            self.play(
                GrowArrow(l_arrow),
                GrowArrow(r_arrow),
                FadeIn(diffuse_label),
                FadeIn(final_chart)
            )
            self.next_slide()
            
            if i == 0:

                old_copies = [title, final_chart]
                title = title.copy()
                final_chart = final_chart.copy()
                for mob in old_copies:
                    self.remove(mob)
                to_remove = self.mobjects
                self.add(title)
                self.play(
                    *[FadeOut(mob) for mob in to_remove],
                    final_chart.animate.next_to(title, DOWN),
                )
                for mob in to_remove:
                    self.remove(mob)
                
                init_chart = final_chart
                self.next_slide()

Manim Community v0.19.0

[09/11/25 09:50:22] INFO     Animation 0 : Partial movie file written in                   ]8;id=558080;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=214856;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurBars/2743010386_722531385_3069                         
                             887955.mp4'                                                                           

[09/11/25 09:50:24] INFO     Animation 1 : Partial movie file written in                   ]8;id=848545;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=521804;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurBars/3790338095_1443166991_493                         
                             200557.mp4'                                                                           

[09/11/25 09:50:30] INFO     Animation 2 : Partial movie file written in                   ]8;id=737594;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=911222;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurBars/3790338095_1501145209_253                         
                             2239013.mp4'                                                                          

[09/11/25 09:50:34] INFO     Animation 3 : Partial movie file written in                   ]8;id=374893;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=736495;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurBars/3790338095_1443166991_413                         
                             6852318.mp4'                                                                          

...

                    INFO     Rendered OurBars                                                          ]8;id=724827;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=819298;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[09/11/25 09:52:42] INFO     Generated 9 slides to                                                      ]8;id=464145;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=781849;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurBars'                             

[09/11/25 09:52:42] INFO     Generated 9 slides to                                                      ]8;id=56114;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=38939;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurBars'                             

                    INFO     Slide 'OurBars' configuration written in                                   ]8;id=237698;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=542245;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurBars.json'                              

                    INFO     Slide 'OurBars' configuration written in                                   ]8;id=409498;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=115121;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurBars.json'                              

# FE LB

## LBSection

In [66]:
%%manim -qh LBSection

class LBSection(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        section = Tex(colorcode(r"\textsc{FunctionErasure} Requires \\ $\Omega(\sqrt{N})$ In-Place Queries"))
        secnum = VGroup(Circle(color=WHITE, radius=0.4), Tex("3")).scale(0.8).next_to(section, UP, buff=0.5)
        self.add(section, secnum)
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/14/25 11:24:08] INFO     Animation 0 : Partial movie file written in                   ]8;id=416629;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=442822;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LBSection/2743010386_722531385_10                         
                             25913328.mp4'                                                                         

[09/14/25 11:24:11] INFO     Animation 1 : Partial movie file written in                   ]8;id=355183;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=460903;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/LBSection/3790338095_1443166991_2                         
                             649319652.mp4'                                                                        

                    INFO     Combining to Movie file.                                      ]8;id=598841;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=985727;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=408664;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=140695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/LBSection.mp4'                                                                
                                                                                                                   

                    INFO     Rendered LBSection                                                        ]8;id=894915;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=474790;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/14/25 11:24:14] INFO     Generated 1 slides to                                                      ]8;id=89170;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=623021;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/LBSection'                           

[09/14/25 11:24:14] INFO     Generated 1 slides to                                                      ]8;id=270202;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=320075;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/LBSection'                           

                    INFO     Slide 'LBSection' configuration written in                                 ]8;id=324787;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=384673;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/LBSection.json'                            

                    INFO     Slide 'LBSection' configuration written in                                 ]8;id=694316;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=448174;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/LBSection.json'                            

## FuncErasToXOR

In [35]:
%%manim -qh FuncErasToXOR

class FuncErasToXOR(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Why is \textsc{FunctionErasure} hard with in-place queries?"), font_size=40)
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r"Proof sketch:",
            r"1) Solving \textsc{FunctionErasure} $\implies$ simulating XOR queries",
            slide=self, align_ref=title
        )
        bullets2 = Bullets(
            r"2) Simulating XOR queries $\implies$ solving \textsc{PermutationInversion}",
            r"3) \textsc{PermutationInversion} requires $\Omega(\sqrt{N})$ in-place queries [FK18][BY23]",
            slide=self, align_ref=bullets
        )
        self.play(
            bullets2.animate.shift(DOWN*3)
        )
        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )
        in_place_defn[0].stretch_to_fit_width(xor_defn[0].width)
        oracle_defns = VGroup(
            xor_defn, 
            in_place_defn
        ).scale(BULLET_TEX_SCALE*0.9).arrange(DOWN).next_to(bullets, DOWN).shift(LEFT*2)
        self.play(FadeIn(oracle_defns))
        self.next_slide()

        steps = Steps(scale_factor=0.7)
        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
        ).next_to(bullets, DOWN).shift(RIGHT*2).play(self)

        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|0\rangle", 
            cause=""
        ).play_diff(self, diff=[2])

        self.remove(steps[-1])
        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|x\rangle",
            cause=r"\text{CNOT}", overlap_prev=True
        ).play_diff(self)
        self.remove(steps[-2])
        steps.remove(steps[-2])

        steps.add_step(
            r"|x\rangle",
            r"|a\rangle",
            r"|f(x)\rangle",
            cause=r"P_f"
        ).play_diff(self)

        steps.add_step(
            r"|x\rangle",
            r"|a \oplus f(x)\rangle",
            r"|f(x)\rangle",
            cause=r"\text{CNOT}"
        ).play_diff(self)

        steps.add_step(
            r"|x\rangle",
            r"|a \oplus f(x)\rangle",
            r"|0\rangle",
            cause=r"\text{FE}"
        ).play_diff(self)
        
        self.play(FadeOut(steps.prev_result()[2]))
        self.next_slide()

Manim Community v0.19.0

[09/14/25 08:48:21] INFO     Animation 0 : Using cached data (hash :                           ]8;id=696167;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=766695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2743010386_722531385_2619554607)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=466528;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=270081;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_805284871)                                                      

[09/14/25 08:48:22] INFO     Animation 2 : Using cached data (hash :                           ]8;id=218062;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=462832;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_2212401791_4077562148)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=557090;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=10999;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_4099341615)                                                     

...

                    INFO     Rendered FuncErasToXOR                                                    ]8;id=909922;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=689656;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 32 animations                                                                  

[09/14/25 08:51:46] INFO     Generated 13 slides to                                                     ]8;id=653340;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=204242;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasToXOR'                       

[09/14/25 08:51:46] INFO     Generated 13 slides to                                                     ]8;id=28432;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=728106;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FuncErasToXOR'                       

                    INFO     Slide 'FuncErasToXOR' configuration written in                             ]8;id=95217;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=790541;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasToXOR.json'                        

                    INFO     Slide 'FuncErasToXOR' configuration written in                             ]8;id=811278;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=361318;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FuncErasToXOR.json'                        

## OurLB

In [39]:
%%manim -qh OurLB

class OurLB(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Simulating an XOR query $\implies$ solving \textsc{PermutationInversion}"), font_size=36)
        self.add(title)
        self.wait()
        self.next_slide()

        # assump = Bullets(r"Imagine for sake of contradiction, $o(\sqrt{N})$ in-place queries suffice:").next_to(title, DOWN)

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )
        in_place_defn[0].stretch_to_fit_width(xor_defn[0].width)

        defns = VGroup(
            xor_defn, 
            in_place_defn
        ).scale(0.7).arrange(DOWN).next_to(title, DOWN*5).align_to(title, LEFT)
        self.play(FadeIn(defns))
        self.next_slide()

        # self.play(FadeIn(assump))
        # self.next_slide()

        ckts = VGroup(
            OracleCircuit(r"P_f", r"|\psi\rangle", r"S_f|\psi\rangle"),
            OracleCircuit(r"P_f", r"|x\rangle|f(x)\rangle", r"|x\rangle|0\rangle"),
            OracleCircuit(r"P_f^{-1}", r"|x\rangle|0\rangle", r"|x\rangle|f(x)\rangle", flipped=True),
            OracleCircuit(r"P_{f^{-1}}", r"|x\rangle|0\rangle", r"|x\rangle|f(x)\rangle", flipped=True),
            OracleCircuit(r"P_f", r"|x\rangle|0\rangle", r"|x\rangle|f^{-1}(x)\rangle", flipped=True),
        ).scale(0.7).next_to(title, DOWN*2.5).align_to(title, RIGHT).shift(RIGHT*0.5)
        self.play(FadeIn(ckts[0][0]), FadeIn(ckts[0][1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckts[0][-1]], run_time=1.2), FadeIn(ckts[0][2], run_time=1.2))
        self.next_slide()
        
        self.play(ReplacementTransform(ckts[0], ckts[1]))
        self.next_slide()
        
        FlipTransform(self, ckts[1], ckts[2])
        self.next_slide()

        for i in range(2,len(ckts) - 1):
            self.play(ReplacementTransform(ckts[i], ckts[i+1]))
            self.next_slide()

        conclusion = Bullets(r"Solves \textsc{PermutationInversion}!").shift(DOWN*2.3)
        self.play(FadeIn(conclusion))
        self.next_slide()

Manim Community v0.19.0

[09/14/25 09:03:19] INFO     Animation 0 : Partial movie file written in                   ]8;id=599593;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=900623;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurLB/2743010386_722531385_169442                         
                             9160.mp4'                                                                             

[09/14/25 09:03:22] INFO     Animation 1 : Partial movie file written in                   ]8;id=229027;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=264425;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurLB/3790338095_1443166991_42104                         
                             12363.mp4'                                                                            

[09/14/25 09:03:26] INFO     Animation 2 : Partial movie file written in                   ]8;id=234184;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=130284;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurLB/3790338095_335623110_336060                         
                             0144.mp4'                                                                             

[09/14/25 09:03:30] INFO     Animation 3 : Partial movie file written in                   ]8;id=472243;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=945731;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/OurLB/3790338095_1443166991_16356                         
                             65529.mp4'                                                                            

...

                    INFO     Rendered OurLB                                                            ]8;id=790322;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=562860;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 19 animations                                                                  

[09/14/25 09:05:12] INFO     Generated 9 slides to                                                      ]8;id=719585;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=137623;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

[09/14/25 09:05:12] INFO     Generated 9 slides to                                                      ]8;id=308899;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=760874;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=247843;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=414780;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=308513;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=924189;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

# Conclusion

## ApplyingAlg

In [47]:
%%manim -qh ApplyingAlg

class ApplyingAlg(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Corollaries"))
        self.add(title)
        self.wait()
        self.next_slide()

        bullets = Bullets(
            r".$\textsc{PermutationInversion}$ takes $\Theta(\sqrt{N})$ in-place queries",
            r".$\textsc{FunctionErasure}$ takes $\Theta(\sqrt{N})$ in-place queries",
            align_ref=title
        ).play(self)
        
        ref = Square().next_to(bullets, DOWN*3).set_x(0)
        ul = ref.get_corner(UL)
        ur = ref.get_corner(UR)
        dl = ref.get_corner(DL)
        dr = ref.get_corner(DR)
        refs = [ul,ur,dl,dr]

        sf = MathTex(colorcode("S_f")).move_to(ul)
        pf = MathTex(colorcode("P_f")).move_to(ur)
        sfi = MathTex(colorcode("S_{f^{-1}}")).move_to(dl)
        pfi = MathTex(colorcode("P_{f^{-1}}")).move_to(dr)

        arrows = VGroup()
        for i in refs:
            for j in refs:
                if (i != j).any():
                    arrows.add(StandardArrow(start=i,end=j,buff=0.6))

        sim = Bullets(
            r"Each requires $\Theta(\sqrt{N})$ queries to simulate the others",
        ).next_to(ref, DOWN*3)
        
        self.play(FadeIn(VGroup(sf,pf,sfi,pfi)))
        self.next_slide()

        self.play(*[GrowArrow(arrow) for arrow in arrows], FadeIn(sim))
        self.next_slide()


Manim Community v0.19.0

[09/11/25 09:57:51] INFO     Animation 0 : Partial movie file written in                   ]8;id=216142;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=956615;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ApplyingAlg/2743010386_722531385_                         
                             584293847.mp4'                                                                        

[09/11/25 09:57:53] INFO     Animation 1 : Partial movie file written in                   ]8;id=605537;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=216374;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ApplyingAlg/3790338095_1443166991                         
                             _1210276159.mp4'                                                                      

[09/11/25 09:57:56] INFO     Animation 2 : Partial movie file written in                   ]8;id=688132;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=268054;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ApplyingAlg/3790338095_2014360137                         
                             _3098501564.mp4'                                                                      

[09/11/25 09:57:59] INFO     Animation 3 : Partial movie file written in                   ]8;id=357673;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=593837;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ApplyingAlg/3790338095_1443166991                         
                             _3636091899.mp4'                                                                      

...

                    INFO     Rendered ApplyingAlg                                                      ]8;id=255752;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=261649;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 10 animations                                                                  

[09/11/25 09:58:28] INFO     Generated 5 slides to                                                      ]8;id=435651;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=144203;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ApplyingAlg'                         

[09/11/25 09:58:28] INFO     Generated 5 slides to                                                      ]8;id=233750;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=864281;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ApplyingAlg'                         

                    INFO     Slide 'ApplyingAlg' configuration written in                               ]8;id=920046;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=685386;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ApplyingAlg.json'                          

                    INFO     Slide 'ApplyingAlg' configuration written in                               ]8;id=980957;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=49644;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ApplyingAlg.json'                          

## FutureWork

In [36]:
%%manim -qh FutureWork

class FutureWork(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Future Work"))
        self.add(title)
        self.wait()
        self.next_slide()

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE)

        bullets = Bullets(
            open_prob,
            1,
            r"Many candidate problems:",
            r".Simon's with garbage [Aar21]",
            r".Embedded permutation inversion",
            1,
            r"Very few available lower bound techniques",
            r".Polynomial method provably fails",
            r".Adversary method requires unprecedented adversary matrices",
            align_ref=title,
        ).play(self)

Manim Community v0.19.0

[09/14/25 08:54:10] INFO     Animation 0 : Using cached data (hash :                           ]8;id=38927;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=514731;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2743010386_722531385_3955229171)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=831756;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=713013;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1443166991_276328905)                                                      

[09/14/25 08:54:11] INFO     Writing \RaggedRight{ Simon's with garbage [Aar21] } to        ]8;id=299255;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=384260;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\
                             media/Tex/5d28de1dffaa3826.tex                                                        

[09/14/25 08:54:13] INFO     Animation 2 : Using cached data (hash :                           ]8;id=912013;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=87599;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3790338095_1765551709_2579437671)                                                     

...

                    INFO     Rendered FutureWork                                                       ]8;id=22062;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=27908;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 16 animations                                                                  

[09/14/25 08:55:00] INFO     Generated 8 slides to                                                      ]8;id=890743;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=116575;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FutureWork'                          

[09/14/25 08:55:00] INFO     Generated 8 slides to                                                      ]8;id=301862;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=767002;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/FutureWork'                          

                    INFO     Slide 'FutureWork' configuration written in                                ]8;id=869345;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=422045;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FutureWork.json'                           

                    INFO     Slide 'FutureWork' configuration written in                                ]8;id=138984;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=196394;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/FutureWork.json'                           

## ThankYou

In [49]:
%%manim -qh ThankYou

class ThankYou(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        takeaway = Tex(colorcode(r"Takeaways:")).scale(1).shift(UP*2)
        takeaway1 = Tex(colorcode(r"Search is possible with $\Theta(\sqrt{N})$ in-place queries.")).scale(0.7).next_to(takeaway, DOWN)
        takeaway2 = Tex(colorcode(r"Some tasks require more in-place queries than XOR queries.")).scale(0.7).next_to(takeaway1, DOWN)
        thanks = Tex(colorcode(r"Thank you!")).scale(2).next_to(takeaway2, DOWN, buff=1.5)

        self.play(FadeIn(takeaway))
        self.next_slide()
        self.play(FadeIn(takeaway1))
        self.next_slide()
        self.play(FadeIn(takeaway2))
        self.next_slide()
        self.play(Write(thanks))
        self.next_slide()


Manim Community v0.19.0

[09/11/25 09:59:37] INFO     Animation 0 : Partial movie file written in                   ]8;id=191537;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=980376;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ThankYou/2743010386_860462169_223                         
                             132457.mp4'                                                                           

[09/11/25 09:59:39] INFO     Animation 1 : Partial movie file written in                   ]8;id=923436;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=226652;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ThankYou/3790338095_1443166991_12                         
                             38007623.mp4'                                                                         

[09/11/25 09:59:42] INFO     Animation 2 : Partial movie file written in                   ]8;id=764994;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=143214;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ThankYou/3790338095_2576878860_10                         
                             46600934.mp4'                                                                         

[09/11/25 09:59:44] INFO     Animation 3 : Partial movie file written in                   ]8;id=774597;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=481570;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             1080p60/partial_movie_files/ThankYou/3790338095_1443166991_35                         
                             9625883.mp4'                                                                          

...

                    INFO     Rendered ThankYou                                                         ]8;id=999159;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=967677;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 8 animations                                                                   

[09/11/25 10:00:09] INFO     Generated 4 slides to                                                      ]8;id=692682;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=148096;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ThankYou'                            

[09/11/25 10:00:09] INFO     Generated 4 slides to                                                      ]8;id=782090;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=956437;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ThankYou'                            

                    INFO     Slide 'ThankYou' configuration written in                                  ]8;id=279626;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=449829;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ThankYou.json'                             

                    INFO     Slide 'ThankYou' configuration written in                                  ]8;id=437390;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=168713;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ThankYou.json'                             

# Purgatory

## XORvsInPlace

In [50]:
#%%manim -qh XORvsInPlace

class XORvsInPlace(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        
        xor_bullets = [
            MathTex(r"\ket{x}\ket{a} \rightarrow \ket{x}\ket{f(x)}", tex_template=myTemplate),
            Text("Any f"),
            Text("Can erase f(x)"),
            Text("Self-inverse"),
            Text("Phase version")
        ]
        inplace_bullets = [
            MathTex(r"\ket{x} \rightarrow \ket{f(x)}", tex_template=myTemplate),
            Text("Bijections"),
            Text("Can erase x")
        ]
        
        xor_group = VGroup(*xor_bullets).arrange(DOWN)
        inplace_group = VGroup(*inplace_bullets).arrange(DOWN)

        tchart = MobjectTable(
            [[xor_group, inplace_group]],
            col_labels=[Text("XOR"), Text("In-Place")],
            arrange_in_grid_config={"row_alignments":'uu'})

        self.play(
            Write(tchart.get_horizontal_lines()), 
            Write(tchart.get_vertical_lines()),
            Write(tchart.get_col_labels()))
        self.next_slide()

        for i in range(len(xor_bullets)):
            self.play(Write(xor_bullets[i]))
            self.next_slide()

            if i < len(inplace_bullets):
                self.play(Write(inplace_bullets[i]))
                self.next_slide()

## Results

In [51]:
# %%manim -qh Results

class Results(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        ### Known Results
        title = Title("Known Results")
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        sep = 1.5
        oracle_defns = VGroup(
            xor_defn, in_place_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep).next_to(title, DOWN*sep)
        prob_defns = VGroup(
            func_erase_defn, idx_erase_defn, perm_inv_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep)
        (prob_defns-func_erase_defn).next_to(oracle_defns, DOWN*sep)
        func_erase_defn.next_to(idx_erase_defn, LEFT*sep)

        self.play(FadeIn(oracle_defns), FadeIn(prob_defns - func_erase_defn)) #TODO animate sequentially

        known_bullets = Bullets(
            r".\textsc{PermutationInversion} requires $\Omega(\sqrt{N})$ queries {\color{X_O_R} [Amb02]}{\color{IN_PLACE} [FK18]}",
            r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ XOR queries {\color{X_O_R} [Amb02]}",
            r"_.{\color{IN_PLACE} [FK18]} Conjecture $\Omega(N)$ in-place queries"
            r".Simulating an in-place query requires $\Omega(\sqrt{N})$ XOR queries",
            r".\textsc{IndexErasure} requires $\Omega(\sqrt{N})$ XOR queries {\color{X_O_R} [AMRR11]}",
            # r".A controlled in-place query can be implemented with one in-place query",
        ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)

        # TODO: known_bullets

        ### Our Results
        our_results_title = Title("Our Results")
        self.play(FadeTransform(title, our_results_title))
        self.next_slide()

        bullets = Bullets(
            r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ in-place queries",
            r".Simulating an XOR query requires $\Omega(\sqrt{N})$ in-place queries",
            r"_$\implies$ \textsc{FunctionErasure} requires $\Omega(\sqrt{N})$ in-place queries",
            # r".A controlled in-place query can be implemented with one in-place query",
        ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)
        
        self.play(FadeIn(bullets[0]))
        self.next_slide()
        
        alg_tight = Tex(r"$\leftarrow$ tight [FK18][BY23]").scale(BULLET_TEX_SCALE*0.8).next_to(bullets[0]).shift(LEFT*0.1)
        self.play(FadeIn(alg_tight))
        self.next_slide()

        self.play(FadeIn(bullets[1]))
        self.next_slide()

        self.play(FadeIn(func_erase_defn), prob_defns.animate.next_to(oracle_defns, DOWN*sep))
        self.play(FadeIn(bullets[2]))
        self.next_slide()

        LBs_tight = VGroup(
            Brace(bullets[1:], RIGHT), 
            Tex(r"tight by above \\ algorithm").scale(BULLET_TEX_SCALE*0.8)
        ).arrange(RIGHT).next_to(bullets[1]).align_to(bullets[1], UP)
        self.play(FadeIn(LBs_tight))
        self.next_slide()

## ComparisonTable

In [52]:
#%%manim -qh ComparisonTable

class ComparisonTable(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        def ul(upper, ucite, lower, lcite):
            u = Tex(rf"$O({upper})$ [{ucite}]")
            l = Tex(rf"$\Omega({lower})$ [{lcite}]").next_to(u, DOWN).align_to(u, LEFT)
            return VGroup(u, l)

        xp_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "Amb02")
        xi_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "AMRR11")
        xf_ul = MathTex(rf"\Theta(1)").scale(2)
        ip_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", "FK18][BY23")
        ii_ul = MathTex(rf"\Theta(1)").scale(2)
        if_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", r"H\textbf{R}Y25")

        ip_ul[0].set_color(RED)
        if_ul.match_color(ip_ul[0])
        func_erase_defn.match_color(ip_ul[0])

        table = VGroup(
            VMobject(), idx_erase_defn, perm_inv_defn, func_erase_defn,
            xor_defn, xi_ul, xp_ul, xf_ul,
            in_place_defn, ii_ul, ip_ul, if_ul,
        )

        table.arrange_in_grid(
            col_alignments="rccc",
            row_alignments="dcc",
        )

        conj = Tex(
            rf"[FK18]: Is $O(N)$ optimal?"
        ).next_to(ip_ul[1], UP).align_to(ip_ul[1], LEFT)

        (table+conj).match_width(title).shift(UP*0.5)
        cross = Cross(conj)

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE).next_to(table, DOWN*2)
        
        def fadein(*x):
            self.play(*[FadeIn(item) for item in x])
            self.next_slide()

        fadein(xor_defn, in_place_defn)
        fadein(idx_erase_defn)
        fadein(ii_ul)
        fadein(xi_ul[1])
        fadein(xi_ul[0])

        fadein(perm_inv_defn)
        fadein(xp_ul)
        fadein(ip_ul[1])
        fadein(conj)
        fadein(cross)
        self.play(FadeOut(conj), FadeOut(cross), FadeIn(ip_ul[0]))
        self.next_slide()

        fadein(func_erase_defn)
        fadein(xf_ul)
        fadein(if_ul[1])
        fadein(if_ul[0])

        fadein(open_prob)

## InvertingCircuits1

In [53]:
#%%manim -qh InvertingCircuits1

class InvertingCircuits1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{A} & \multigate{1}{C} & \gate{D} & \qw \\
            & \gate{B} & \ghost{C} & \gate{E} & \qw
        }'''
        abcde_ckt = MathTex(abcde_tex, tex_template=myTemplate)
        abcde_braces = CircuitBraces(abcde_ckt, r"|\psi\rangle", r"\mathcal{C}|\psi\rangle")
        abcde = VGroup(abcde_ckt, abcde_braces)

        self.play(Write(abcde))
        self.next_slide()

        inv_abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{D^{-1}} & \multigate{1}{C^{-1}} & \gate{A^{-1}} & \qw \\
            & \gate{E^{-1}} & \ghost{C^{-1}} & \gate{B^{-1}} & \qw
        }'''
        inv_abcde_ckt = MathTex(inv_abcde_tex, tex_template=myTemplate)
        inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"\mathcal{C} |\psi\rangle", r"|\psi\rangle")
        inv_abcde = VGroup(inv_abcde_ckt, inv_abcde_braces)

        FlipTransform(self, abcde, inv_abcde)
        self.next_slide()

        new_inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"|\psi\rangle", r"\mathcal{C}^{-1} |\psi\rangle")
        self.play(Transform(inv_abcde_braces, new_inv_abcde_braces))
        # self.play(*[TransformMatchingShapes(inv_abcde_braces[i], new_inv_abcde_braces[i]) for i in (1,3)])
        self.next_slide()
        

## Kashefi

In [54]:
#%%manim -qh Kashefi

class Kashefi(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"S_f", r"|\psi\rangle", r"P_f|\psi\rangle")
        ckt_f = OracleCircuit(r"S_f", r"|x\rangle", r"|f(x)\rangle")

        ckt_flipped = OracleCircuit(r"S_f^{-1}", r"|f(x)\rangle", r"|x\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"S_f^{-1}", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"S_f", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()
        self.play(ReplacementTransform(ckt, ckt_f))
        self.next_slide()

        FlipTransform(self, ckt_f, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $S_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

## AlgCkt

In [55]:
#%%manim -qh AlgCircuit

class AlgCircuit(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # qcircuit
        qcircuit = r'''
        \Qcircuit @C=.5em @R=0.5em @!R {
         & & & & & & \mbox{\hspace{1.8em}\textit{Mark}} & & & & & & & & \mbox{\textit{Shift}} & & & & & & & & \mbox{\textit{Diffuse the Difference}} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \ctrl{4}  & \qw       & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \multigate{2}{\Ppi} & \ctrl{3}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \multigate{2}{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \ctrl{4}  & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \ctrl{2}  & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \rstick{\ket{\psi_{t}}_\regA} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \ctrl{4}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \qw       & \ctrl{1}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}
    \inputgroupv{2}{4}{0.5em}{2em}{\ket{\psi_{t-1}}_\regA \hspace{1.3em}} \\
    	& \lstick{\ket{0}_\regC \Big\{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ctrlo{-1} & \ctrlo{1} & \ctrlo{2} & \ctrlo{3} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \gate{H}  & \ctrl{-1} & \gate{H}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \meter    & \cw       & \rstick{\text{\textit{Measure} }\regC} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \targ     & \qw       & \qw       & \multigate{2}{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \rstick{\ket{0^n}_\regB} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}
    \inputgroupv{6}{8}{0.5em}{2em}{\ket{0^n}_\regB \hspace{0.5em}}
    \gategroup{2}{29}{4}{29}{0em}{\}}
    \gategroup{6}{21}{8}{21}{0em}{\}}
    \gategroup{2}{4}{8}{10}{0.7em}{--}
    \gategroup{2}{13}{8}{18}{0.7em}{--}
    \gategroup{2}{21}{5}{25}{0.7em}{--}\\
    }
        '''
        label = MathTex(qcircuit, tex_template=myTemplate).scale(0.5)
        self.play(Write(label))
        self.wait()
        self.next_slide()

## StepThruOurAlgBar

In [56]:
#%%manim -qh StepThruOurAlgBar

class StepThruOurAlgBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/2 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        self.play(DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        for t in range(1,T):
            a_t,b_t = get_HRY_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            self.play(chart.animate.change_bar_values(values))
            self.next_slide()


## [OLD] Our Alg

In [57]:
#%%manim -qh OurAlgOLD

QUICK_RENDER_STEPS = True

class OurAlgOLD(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Can we mimic one iteration of Grover's search?"))
        self.add(title)
        self.next_slide()

        tech_title = Title(colorcode(r"Can we amplify $|x^*\rangle$ by $\Omega\left( \tfrac{1}{\sqrt{N}} \right)$ with $O(1)$ in-place queries?"), font_size=36)
        self.play(Transform(title, tech_title))
        self.next_slide()

        # bullets = Bullets(
        #     r"Can we mimic one iteration of Grover's algorithm?",
        #     1,
        #     r"In one iteration, we need to:",
        #     r".increase amplitude on $|x^*\rangle$ by $O\left(\frac{1}{\sqrt{N}}\right)$ \\ (Recall $x^* := f^{-1}(1)$ is our marked item)",
        #     r".use a constant number of in-place queries",
        #     font_scale=0.9, align_ref=title
        # )
        ## bullets[3][1][0][20:28].set_color(IN_PLACE)
        # bullets.play(self)

        steps = Steps(scale_factor=0.5)
        steps.add_step(r"\sum_{x \in [N]} |x\rangle", r"|0\rangle").shift(2*UP+2*RIGHT)
        steps.play(self, indices=[(-1,-1,0)])
        steps.play(self, indices=[(-1,-1,1)])
        first_step = steps.prev_result()[0]
        steps.add_step(r"\sum_{x \in [N]} |x\rangle |x\rangle", cause=r"\text{CNOT}").play(self)
        steps.add_step(r"\sum_{x \in [N]}", r"|x\rangle |f(x)\rangle", cause=r"P_f").play(self)

        first_bad_step = len(steps)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |f(x)\rangle").play(self)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |0\rangle", cause=r"\text{???}").play(self)

        funcEras = MathTex(
            r"\xrightarrow[\text{Erasure}]{\text{Function}}"
        ).scale(steps.scale_factor).next_to(steps.prev_result(), LEFT).set_color(RED)
        self.play(ReplacementTransform(steps[-1][0], funcEras))
        self.next_slide()

        cross = Cross(steps[-1])
        self.play(Create(cross))
        self.next_slide()

        steps.add(cross)
        steps.play(self, indices=range(first_bad_step, len(steps)), animation=FadeOut)
        for _ in range(first_bad_step, len(steps)):
            steps.remove(steps[-1])
        self.next_slide()
        
        premark = MathTex(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle", 
            r" + |x^*\rangle |1\rangle"
        ).scale(steps.scale_factor).next_to(steps[-2][-1], DOWN).align_to(steps[-2][-1], LEFT)
        self.play(
            FadeOut(steps.prev_result()[0], rate_func=slow_into), 
            FadeIn(premark[0], rate_func=slow_into),
            FadeTransform(steps.prev_result()[1], premark[1]),
            TransformFromCopy(steps.prev_result()[1], premark[2]),
        )
        steps[-1].remove(steps[-1][-1])
        steps[-1].add(premark)
        self.next_slide()

        # Mark
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle"
        ).play(self)

        # Shift
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle",
            cause = r"\text{ctrl-}P_f"
        ).play(self)

        # Cleanup
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle", 
            r"|0\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"|0\rangle", 
            r"\otimes |1\rangle",
            cause = r"\text{CNOT}"
        ).play(self)
        clean = steps.prev_result()

        cleaner = MathTex(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        ).scale(steps.scale_factor).align_to(clean, UP).align_to(clean, LEFT)

        cleanest = MathTex(
            r"\sum_{\substack{x \in [N] \\ x \ne 1}}",
            r"|x\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        ).scale(steps.scale_factor).align_to(clean, UP).align_to(clean, LEFT)

        self.play(
            FadeOut(clean[2]),
            FadeOut(clean[4])
        )
        clean.remove(clean[4])
        clean.remove(clean[2])
        self.play(
            TransformMatchingTex(clean, cleaner)
        )
        self.next_slide()
        for i in range(len(clean)):
            self.remove(clean[i])
        self.remove(clean)

        self.play(
            FadeTransform(cleaner[0], cleanest[0]),
            *[Transform(cleaner[i], cleanest[i]) for i in range(1,4)]
        )
        self.next_slide()

        steps[-1][-1] = cleanest

        marknshift = MathTex(
            r"\sum_{x \in [N]} |x\rangle",
            r"\rightarrow",
            r"\sum_{\substack{x \in [N] \\ x \ne 1}}",
            r"|x\rangle", 
            r"\otimes |0\rangle + |x^*\rangle", 
            r"\otimes |1\rangle",
        )

        for i in range(1,4):
            self.remove(cleaner[i])

        self.play(
            FadeOut(steps),
            TransformFromCopy(first_step, marknshift[0]),
            TransformFromCopy(steps[-1][-1], marknshift[2:]),
        )
        self.play(FadeIn(marknshift[1]))
        self.next_slide()

## StaticComparisonTable

In [58]:
#%%manim -qh StaticComparisonTable

class StaticComparisonTable(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.wait()
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        def ul(upper, ucite, lower, lcite):
            u = Tex(rf"$O({upper})$ [{ucite}]")
            l = Tex(rf"$\Omega({lower})$ [{lcite}]").next_to(u, DOWN).align_to(u, LEFT)
            return VGroup(u, l)

        xp_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "Amb02")
        xi_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "AMRR11")
        xf_ul = MathTex(rf"\Theta(1)").scale(2)
        ip_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", "FK18][BY23")
        ii_ul = MathTex(rf"\Theta(1)").scale(2)
        if_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", r"H\textbf{R}Y25")

        ip_ul[0].set_color(RED)
        if_ul.match_color(ip_ul[0])
        func_erase_defn.match_color(ip_ul[0])

        table = VGroup(
            VMobject(), idx_erase_defn, perm_inv_defn, func_erase_defn,
            xor_defn, xi_ul, xp_ul, xf_ul,
            in_place_defn, ii_ul, ip_ul, if_ul,
        )

        table.arrange_in_grid(
            col_alignments="rccc",
            row_alignments="dcc",
        )

        conj = Tex(
            rf"[FK18]: Is $O(N)$ optimal?"
        ).next_to(ip_ul[1], UP).align_to(ip_ul[1], LEFT)

        (table+conj).match_width(title).shift(UP*0.5)
        cross = Cross(conj)

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE).next_to(table, DOWN*2)

        thankyou = Tex(
            colorcode(r"Thank you!")
        ).next_to(open_prob, DOWN*2)
        
        def fadein(*x):
            self.play(*[FadeIn(item) for item in x])
            self.next_slide()

        self.add(table)
        self.wait()
        self.next_slide()
        
        fadein(open_prob)
        
        self.play(Write(thankyou))
        self.next_slide()